# Otimização de hiperparâmetros


- Objetivo: Utilizar a otimização bayesiana de hiperparâmetros nos modelos de leves

# Pacotes

In [0]:
!pip install shap

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from pyspark.sql.functions import *
from pyspark.sql.functions import expr
from pyspark.sql.types import ByteType,ShortType,LongType,IntegerType,DoubleType
import seaborn as sns
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, SelectKBest, chi2
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
import statsmodels.api as sm

# Classes

In [0]:
%run /Users/felipe.espirito@bv.com.br/Modulos

# Funções

In [0]:
%run /Users/lucas.lima@bv.com.br/funcoes/funcoes_lucas

In [0]:
%run /Users/felipe.espirito@bv.com.br/funcoes

tabela_mmNtx(nomebase,nvar,nfxvar,ntarget) | última atualização: 16/03/2022

Out[3]: True

calc_perf(n_base,v_score,v_target) | última atualização: 03/03/2022

[1] "calcKS_R <- function(n_target,n_score)"

cria_fxs(nbase1,nbase2,nvar,n_fxs,nova_var) | última atualização 08/03/2022

calc_IV(nbase1,nvar_fx,ntarget) | última atualização 11/03/2022

aplicaRUPD(nbase,nvar) | Última atualização: 30/03/2022

encontra_modelo(X, y, classificador, n_divisoes, n_repeticoes,hiperparams = None, scoring="accuracy", random_state=1109) 
 Última atualização: 21/03/2022

# Dados

In [0]:
%sql 
refresh table cred_lucaslima.levesFast_escorada_ncli;
select * from cred_lucaslima.levesFast_escorada_ncli limit 10;

NuProposta,NuCpfCnpj,DtPrimeiroCadastro,QtMesesEmprego,DsEstadoCivil,CdUfCliente,CPF_main,Safra_main,P3_ult_ret,P4_BVS,SCORE_HRIM,SCORE_HSPN,SCORE_HVLD,QuodScore_v2,score_ajust,inad30,elegiveis,QuodScore,Blend_Novo_1_1_arred,RUPD,ScoreBacen,WoE_cd_mosaic,WoE_score_ajust,WoE_mpv_score,WoE_pb_qt_propavgdiaauto_pv,WoE_pb_qt_diasfina_max_pv,WoE_pb_td_primcompr_pv,WoE_pb_td_ultcomprfin_pv,WoE_PF_CP_VOL04_PF,PF_CP_VOL04_PF,WoE_PP_PD_CPG01_PF,PP_PD_CPG01_PF,WoE_PF_CP_VOL03_PF,PF_CP_VOL03_PF,WoE_PF_CP_VOL02_PF,PF_CP_VOL02_PF,WoE_RM_PR_MOD04_PF,RM_PR_MOD04_PF,WoE_PF_CP_VOL01_PF,PF_CP_VOL01_PF,WoE_RM_PR_ETB02_PF,RM_PR_ETB02_PF,WoE_PF_PC_CPR02_PF,PF_PC_CPR02_PF,WoE_HRIM,yhat1,yhat1_2,yhat1_3,yhat1_4,yhat1_5,logito_CRISP,logito_yhat1_4,yhat_blend,yhat_blend2,yhat_dt,yhat_dt1_1,yhat_rf1_1,qsyhat1,qsyhat1_2,qsyhat1_3,qsyhat1_4,qsyhat1_5,qsyhat_rf1_1,qsyhat_dt1_1,qsyhat_blend,qsyhat_blend2,qsP3_ult_ret,qsP4_BVS,qsSCORE_HRIM,qsSCORE_HSPN,qsSCORE_HVLD,qsQuodScore_v2,qsQuodScore,qsBlend_Novo_1_1_arred,prob,prob90m12,yhat_blend_90m12
661880993,14884097572,-99999999,48,CASADO,BA,14884097572,202108,815.0,541.0,712.0,712.0,972.0,946,-9.9999999E7,0,1,966.0,970.0,7.25,-9.9999999E7,-0.322173,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,0.222394,82.86,0.143172,93.75,0.127051,71.82,0.162961,75.39,0.258504,91.67,0.103328,4.52,0.197557,53.79,0.168483,89.6,0.095048,968.7114457831641,973.9716036466805,972.5219283965179,972.5647792010129,982.3235909440401,3.4760986898352733,3.5681090657224437,973.0425001201331,973.3221918786007,0.027680730986294005,972.319269013706,975.8125835103015,4.0,5.0,4.0,4.0,8.0,6,4,4.0,4.0,7.0,5.0,4.0,4.0,6.0,8,9.0,6.0,0.026957499879866842,0.08158083778213898,918.4191622178611
561291750,15754389760,-99999999,36,SOLTEIRO,ES,15754389760,202108,797.0,597.0,785.0,785.0,943.0,895,644.0,0,1,899.0,927.0,5.0,-9.9999999E7,-0.322173,0.064299,0.183219,0.092449,0.093345,-0.065547,0.206627,-0.135236,35.28,0.333876,100.0,0.127051,77.48,0.162961,81.59,-0.012762,75.0,0.279543,19.07,0.325496,60.36,0.168483,93.27,0.392442,979.3334604278095,979.0573552156699,980.9990661824082,980.4489019976164,974.1760175519048,2.541494124417464,3.914979081389507,979.826883265295,979.5069354072739,0.016415868673050615,983.5841313269493,980.1770238841283,6.0,6.0,7.0,6.0,5.0,7,6,6.0,6.0,6.0,7.0,6.0,6.0,3.0,7,8.0,3.0,0.02017311673470501,0.06176100969129661,938.2389903087034
142116612,16802418851,-99999999,60,CASADO,SP,16802418851,202108,835.0,530.0,817.0,817.0,987.0,662,597.0,0,1,618.0,994.0,9.0,-9.9999999E7,0.24457,0.064299,-0.059677,0.257332,0.179955,0.329513,0.206627,-0.521438,-1.0,-0.545421,-1.0,-0.480301,-1.0,-0.509439,-1.0,-0.495893,-1.0,-0.487331,-1.0,-0.508048,-1.0,-0.491549,-1.0,0.392442,976.0430178567755,960.8351212845394,969.8517529583528,971.0578824919014,960.6014623834286,5.109977737428519,3.5130881903006417,973.2829515094704,972.7410340669836,0.032038007065416006,967.9619929345839,969.7135084589131,5.0,2.0,4.0,4.0,2.0,4,3,4.0,4.0,8.0,5.0,7.0,7.0,8.0,1,0.0,9.0,0.026717048490529582,0.08087286627999934,919.1271337200006
702018028,22548841820,-99999999,144,CASADO,SP,22548841820,202108,715.0,437.0,657.0,657.0,859.0,849,-9.9999999E7,0,1,849.0,787.0,2.5,-9.9999999E7,0.005647,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,0.222394,62.2,0.333876,100.0,-0.103101,38.66,-0.057811,54.0,-0.012762,66.67,-0.05134,2.04,-0.508048,39.19,-0.491549,0.0,0.018759,960.4990607688545,960.8706722719262,960.9401694577533,959.5025790715032,968.5939260807334,1.3069360827079781,3.165176712069993,957.6288905228157,957.6913698780169,0.027680730986294005,972.319269013706,970.0709588007398,2.0,2.0,2.0,2.0,3.0,4,4,2.0,2.0,2.0,2.0,3.0,3.0,0.0,5,5.0,0.0,0.042371109477184254,0.12742398313270176,872.5760168672982
531972125,22744347272,-99999999,24,CASADO,PA,22744347272,202108,695.0,474.0,3.0,435.0,829.0,825,242.0,0,1,618.0,847.0,3.0,-9.9999999E7,0.032051,-0.172671,0.00945,-0.138807,-0.057576,0.082153,-0.047612,-0.521438,-1.0,-0.545421,-1.0,-0.480301,-1.0,-0.509439,-1.0,

In [0]:
base_ncli=spark.sql('''
  select * from cred_lucaslima.levesFast_escorada_ncli
  ''')
print('Base Não Clientes: ', base_ncli.count())

base_ncli_pd=base_ncli.toPandas()

Base Não Clientes: 150969

In [0]:
base_ncli_pd

,NuProposta,NuCpfCnpj,DtPrimeiroCadastro,QtMesesEmprego,DsEstadoCivil,CdUfCliente,CPF_main,Safra_main,P3_ult_ret,P4_BVS,SCORE_HRIM,SCORE_HSPN,SCORE_HVLD,QuodScore_v2,score_ajust,inad30,elegiveis,QuodScore,Blend_Novo_1_1_arred,RUPD,ScoreBacen,WoE_cd_mosaic,WoE_score_ajust,WoE_mpv_score,WoE_pb_qt_propavgdiaauto_pv,WoE_pb_qt_diasfina_max_pv,WoE_pb_td_primcompr_pv,WoE_pb_td_ultcomprfin_pv,WoE_PF_CP_VOL04_PF,PF_CP_VOL04_PF,WoE_PP_PD_CPG01_PF,PP_PD_CPG01_PF,WoE_PF_CP_VOL03_PF,PF_CP_VOL03_PF,WoE_PF_CP_VOL02_PF,PF_CP_VOL02_PF,WoE_RM_PR_MOD04_PF,RM_PR_MOD04_PF,WoE_PF_CP_VOL01_PF,PF_CP_VOL01_PF,WoE_RM_PR_ETB02_PF,RM_PR_ETB02_PF,WoE_PF_PC_CPR02_PF,PF_PC_CPR02_PF,WoE_HRIM,yhat1,yhat1_2,yhat1_3,yhat1_4,yhat1_5,logito_CRISP,logito_yhat1_4,yhat_blend,yhat_blend2,yhat_dt,yhat_dt1_1,yhat_rf1_1,qsyhat1,qsyhat1_2,qsyhat1_3,qsyhat1_4,qsyhat1_5,qsyhat_rf1_1,qsyhat_dt1_1,qsyhat_blend,qsyhat_blend2,qsP3_ult_ret,qsP4_BVS,qsSCORE_HRIM,qsSCORE_HSPN,qsSCORE_HVLD,qsQuodScore_v2,qsQuodScore,qsBlend_Novo_1_1_arred,prob,prob90m12,yhat_blend_90m12
0,661880993,14884097572,-99999999,48,CASADO,BA,14884097572,202108,815.0,541.0,712.0,712.0,972.0,946,-99999999.0,0,1,966.0,970.0,7.25,-99999999.0,-0.322173,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,0.222394,82.86,0.143172,93.75,0.127051,71.82,0.162961,75.39,0.258504,91.67,0.103328,4.52,0.197557,53.79,0.168483,89.60,0.095048,968.711446,973.971604,972.521928,972.564779,982.323591,3.476099,3.568109,973.042500,973.322192,0.027681,972.319269,975.812584,4.0,5.0,4.0,4.0,8.0,6,4,4.0,4.0,7.0,5.0,4.0,4.0,6.0,8,9.0,6.0,0.026957,0.081581,918.419162
1,561291750,15754389760,-99999999,36,SOLTEIRO,ES,15754389760,202108,797.0,597.0,785.0,785.0,943.0,895,644.0,0,1,899.0,927.0,5.00,-99999999.0,-0.322173,0.064299,0.183219,0.092449,0.093345,-0.065547,0.206627,-0.135236,35.28,0.333876,100.00,0.127051,77.48,0.162961,81.59,-0.012762,75.00,0.279543,19.07,0.325496,60.36,0.168483,93.27,0.392442,979.333460,979.057355,980.999066,980.448902,974.176018,2.541494,3.914979,979.826883,979.506935,0.016416,983.584131,980.177024,6.0,6.0,7.0,6.0,5.0,7,6,6.0,6.0,6.0,7.0,6.0,6.0,3.0,7,8.0,3.0,0.020173,0.061761,938.238990
2,142116612,16802418851,-99999999,60,CASADO,SP,16802418851,202108,835.0,530.0,817.0,817.0,987.0,662,597.0,0,1,618.0,994.0,9.00,-99999999.0,0.244570,0.064299,-0.059677,0.257332,0.179955,0.329513,0.206627,-0.521438,-1.00,-0.545421,-1.00,-0.480301,-1.00,-0.509439,-1.00,-0.495893,-1.00,-0.487331,-1.00,-0.508048,-1.00,-0.491549,-1.00,0.392442,976.043018,960.835121,969.851753,971.057882,960.601462,5.109978,3.513088,973.282952,972.741034,0.032038,967.961993,969.713508,5.0,2.0,4.0,4.0,2.0,4,3,4.0,4.0,8.0,5.0,7.0,7.0,8.0,1,0.0,9.0,0.026717,0.080873,919.127134
3,702018028,22548841820,-99999999,144,CASADO,SP,22548841820,202108,715.0,437.0,657.0,657.0,859.0,849,-99999999.0,0,1,849.0,787.0,2.50,-99999999.0,0.005647,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,0.222394,62.20,0.333876,100.00,-0.103101,38.66,-0.057811,54.00,-0.012762,66.67,-0.051340,2.04,-0.508048,39.19,-0.491549,0.00,0.018759,960.499061,960.870672,960.940169,959.502579,968.593926,1.306936,3.165177,957.628891,957.691370,0.027681,972.319269,970.070959,2.0,2.0,2.0,2.0,3.0,4,4,2.0,2.0,2.0,2.0,3.0,3.0,0.0,5,5.0,0.0,0.042371,0.127424,872.576017
4,531972125,22744347272,-99999999,24,CASADO,PA,22744347272,202108,695.0,474.0,3.0,435.0,829.0,825,242.0,0,1,618.0,847.0,3.00,-99999999.0,0.032051,-0.172671,0.009450,-0.138807,-0.057576,0.082153,-0.047612,-0.521438,-1.00,-0.545421,-1.00,-0.480301,-1.00,-0.509439,-1.00,-0.495893,-1.00,-0.487331,-1.00,-0.508048,-1.00,-0.491549,-1.00,-0.724418,944.442532,955.435922,940.356534,937.569275,929.630974,1.711263,2.709233,936.823190,937.065905,0.048065,951.935484,951.828275,1.0,2.0,1.0,0.0,0.0,1,1,0.0,0.0,1.0,3.0,0.0,0.0,0.0,3,0.0,1.0,0.063177,0.189594,810.405512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

# Conversões

In [0]:
base_ncli_pd["Safra_main"] = pd.to_datetime(base_ncli_pd["Safra_main"].astype(str), format='%Y%m')

In [0]:
base_ncli_cat=base_ncli_pd[(base_ncli_pd.Safra_main < "2021-11")]
base_ncli_cat=base_ncli_cat.fillna(-99999999)

base_ncli_cat=base_ncli_cat.dropna(subset=['P3_ult_ret','P4_BVS','SCORE_HRIM',
                                           'SCORE_HVLD','QuodScore_v2','QuodScore'])

base_ncli_cat.head()

,NuProposta,NuCpfCnpj,DtPrimeiroCadastro,QtMesesEmprego,DsEstadoCivil,CdUfCliente,CPF_main,Safra_main,P3_ult_ret,P4_BVS,SCORE_HRIM,SCORE_HSPN,SCORE_HVLD,QuodScore_v2,score_ajust,inad30,elegiveis,QuodScore,Blend_Novo_1_1_arred,RUPD,ScoreBacen,WoE_cd_mosaic,WoE_score_ajust,WoE_mpv_score,WoE_pb_qt_propavgdiaauto_pv,WoE_pb_qt_diasfina_max_pv,WoE_pb_td_primcompr_pv,WoE_pb_td_ultcomprfin_pv,WoE_PF_CP_VOL04_PF,PF_CP_VOL04_PF,WoE_PP_PD_CPG01_PF,PP_PD_CPG01_PF,WoE_PF_CP_VOL03_PF,PF_CP_VOL03_PF,WoE_PF_CP_VOL02_PF,PF_CP_VOL02_PF,WoE_RM_PR_MOD04_PF,RM_PR_MOD04_PF,WoE_PF_CP_VOL01_PF,PF_CP_VOL01_PF,WoE_RM_PR_ETB02_PF,RM_PR_ETB02_PF,WoE_PF_PC_CPR02_PF,PF_PC_CPR02_PF,WoE_HRIM,yhat1,yhat1_2,yhat1_3,yhat1_4,yhat1_5,logito_CRISP,logito_yhat1_4,yhat_blend,yhat_blend2,yhat_dt,yhat_dt1_1,yhat_rf1_1,qsyhat1,qsyhat1_2,qsyhat1_3,qsyhat1_4,qsyhat1_5,qsyhat_rf1_1,qsyhat_dt1_1,qsyhat_blend,qsyhat_blend2,qsP3_ult_ret,qsP4_BVS,qsSCORE_HRIM,qsSCORE_HSPN,qsSCORE_HVLD,qsQuodScore_v2,qsQuodScore,qsBlend_Novo_1_1_arred,prob,prob90m12,yhat_blend_90m12
0,661880993,14884097572,-99999999,48,CASADO,BA,14884097572,2021-08-01,815.0,541.0,712.0,712.0,972.0,946,-99999999.0,0,1,966.0,970.0,7.25,-99999999.0,-0.322173,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,0.222394,82.86,0.143172,93.75,0.127051,71.82,0.162961,75.39,0.258504,91.67,0.103328,4.52,0.197557,53.79,0.168483,89.60,0.095048,968.711446,973.971604,972.521928,972.564779,982.323591,3.476099,3.568109,973.042500,973.322192,0.027681,972.319269,975.812584,4.0,5.0,4.0,4.0,8.0,6,4,4.0,4.0,7.0,5.0,4.0,4.0,6.0,8,9.0,6.0,0.026957,0.081581,918.419162
1,561291750,15754389760,-99999999,36,SOLTEIRO,ES,15754389760,2021-08-01,797.0,597.0,785.0,785.0,943.0,895,644.0,0,1,899.0,927.0,5.00,-99999999.0,-0.322173,0.064299,0.183219,0.092449,0.093345,-0.065547,0.206627,-0.135236,35.28,0.333876,100.00,0.127051,77.48,0.162961,81.59,-0.012762,75.00,0.279543,19.07,0.325496,60.36,0.168483,93.27,0.392442,979.333460,979.057355,980.999066,980.448902,974.176018,2.541494,3.914979,979.826883,979.506935,0.016416,983.584131,980.177024,6.0,6.0,7.0,6.0,5.0,7,6,6.0,6.0,6.0,7.0,6.0,6.0,3.0,7,8.0,3.0,0.020173,0.061761,938.238990
2,142116612,16802418851,-99999999,60,CASADO,SP,16802418851,2021-08-01,835.0,530.0,817.0,817.0,987.0,662,597.0,0,1,618.0,994.0,9.00,-99999999.0,0.244570,0.064299,-0.059677,0.257332,0.179955,0.329513,0.206627,-0.521438,-1.00,-0.545421,-1.00,-0.480301,-1.00,-0.509439,-1.00,-0.495893,-1.00,-0.487331,-1.00,-0.508048,-1.00,-0.491549,-1.00,0.392442,976.043018,960.835121,969.851753,971.057882,960.601462,5.109978,3.513088,973.282952,972.741034,0.032038,967.961993,969.713508,5.0,2.0,4.0,4.0,2.0,4,3,4.0,4.0,8.0,5.0,7.0,7.0,8.0,1,0.0,9.0,0.026717,0.080873,919.127134
3,702018028,22548841820,-99999999,144,CASADO,SP,22548841820,2021-08-01,715.0,437.0,657.0,657.0,859.0,849,-99999999.0,0,1,849.0,787.0,2.50,-99999999.0,0.005647,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,-0.338559,0.222394,62.20,0.333876,100.00,-0.103101,38.66,-0.057811,54.00,-0.012762,66.67,-0.051340,2.04,-0.508048,39.19,-0.491549,0.00,0.018759,960.499061,960.870672,960.940169,959.502579,968.593926,1.306936,3.165177,957.628891,957.691370,0.027681,972.319269,970.070959,2.0,2.0,2.0,2.0,3.0,4,4,2.0,2.0,2.0,2.0,3.0,3.0,0.0,5,5.0,0.0,0.042371,0.127424,872.576017
4,531972125,22744347272,-99999999,24,CASADO,PA,22744347272,2021-08-01,695.0,474.0,3.0,435.0,829.0,825,242.0,0,1,618.0,847.0,3.00,-99999999.0,0.032051,-0.172671,0.009450,-0.138807,-0.057576,0.082153,-0.047612,-0.521438,-1.00,-0.545421,-1.00,-0.480301,-1.00,-0.509439,-1.00,-0.495893,-1.00,-0.487331,-1.00,-0.508048,-1.00,-0.491549,-1.00,-0.724418,944.442532,955.435922,940.356534,937.569275,929.630974,1.711263,2.709233,936.823190,937.065905,0.048065,951.935484,951.828275,1.0,2.0,1.0,0.0,0.0,1,1,0.0,0.0,1.0,3.0,0.0,0.0,0.0,3,0.0,1.0,0.063177,0.189594,810.405512


In [0]:
vars_WoE=list(base_ncli_cat.filter(like='WoE').columns)

X_list=vars_WoE+['P3_ult_ret','P4_BVS','SCORE_HSPN','QuodScore_v2','QuodScore']
X_list

# Lista com variáveis contínuas, com exceção das da B3, que são categóricas.
lista_b3_woe=['WoE_mpv_score','WoE_pb_qt_propavgdiaauto_pv',
              'WoE_pb_qt_diasfina_max_pv','WoE_pb_td_primcompr_pv']

# variáveis adicionais
X_list3 = X_list 

y=base_ncli_cat.inad30

X1=base_ncli_cat[X_list]
X3=base_ncli_cat[X_list3]
X_geral = base_ncli_cat

# Otimização Não Clientes

In [0]:
X=X_geral

prefixos = []
safras_dev = [safra for safra in base_ncli_pd.Safra_main.unique() if safra < pd.to_datetime("2021-11")]
safras_teste = [safra for safra in base_ncli_pd.Safra_main.unique() if safra > pd.to_datetime("2021-11") and safra < pd.to_datetime("2022-01")]
safras_nov = [safra for safra in base_ncli_pd.Safra_main.unique() if safra == pd.to_datetime("2021-11")]

variaveis_PF = [col for col in base_ncli_pd.columns if "WoE_PF" in col]
variaveis_continuas = [col for col in base_ncli_pd.columns if "WoE" not in col and "_PF" in col]

restrito = [
 'WoE_score_ajust',
 'WoE_PF_CP_VOL04_PF',
 'P4_BVS',
 'QuodScore_v2',
 'SCORE_HSPN'
]

aberto = [
  *variaveis_PF,
 'WoE_score_ajust',
 'P4_BVS',
 'QuodScore_v2',
 'SCORE_HSPN'
]

aberto_continuo = [
  *variaveis_continuas,
  'score_ajust',
  'P4_BVS',
  'QuodScore_v2',
  'SCORE_HSPN'
]

X_restrito=X[restrito]
X_aberto=X[aberto]
X_aberto_continuo=X[aberto_continuo]

vis = Visualizador(base=base_ncli_pd, base_cat=base_ncli_cat)
random_state = 1505 # seed geral

## Bayesiana

### AdaBoost

#### Restrito

In [0]:
prefixo = "ab_restrito"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="adaboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_restrito, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:16<26:40, 16.16s/trial, best loss: -0.6660663458818242] 2%|▏ | 2/100 [00:19<13:45, 8.43s/trial, best loss: -0.6660663458818242] 3%|▎ | 3/100 [00:21<08:53, 5.50s/trial, best loss: -0.6660663458818242] 4%|▍ | 4/100 [00:24<07:13, 4.52s/trial, best loss: -0.6687060037919569] 5%|▌ | 5/100 [00:28<06:52, 4.34s/trial, best loss: -0.6691047999906786] 7%|▋ | 7/100 [00:29<03:42, 2.39s/trial, best loss: -0.6691047999906786] 8%|▊ | 8/100 [00:31<03:30, 2.29s/trial, best loss: -0.6691047999906786] 9%|▉ | 9/100 [00:33<03:21, 2.22s/trial, best loss: -0.6691047999906786] 10%|█ | 10/100 [00:36<03:39, 2.44s/trial, best loss: -0.6691047999906786] 13%|█▎ | 13/100 [00:38<02:08, 1.47s/trial, best loss: -0.6696749747791403] 14%|█▍ | 14/100 [00:41<02:34, 1.79s/trial, best loss: -0.6696749747791403] 16%|█▌ | 16/100 [00:43<02:06, 1.50s/trial, best loss: -0.6696749747791403] 18%|█▊ | 18/100 [00:44<01:34, 1.16s/trial, best loss: -0.6696749747791403] 19%|█▉ | 19/100 [00:46<01:47, 1.33s/trial, best loss: -0.6696749747791403] 20%|██ | 20/100 [00:48<01:58, 1.48s/trial, best loss: -0.6696749747791403] 22%|██▏ | 22/100 [00:49<01:25, 1.10s/trial, best loss: -0.6696749747791403] 24%|██▍ | 24/100 [00:50<01:07, 1.13trial/s, best loss: -0.6696749747791403] 25%|██▌ | 25/100 [00:52<01:23, 1.12s/trial, best loss: -0.6700077036377685] 26%|██▌ | 26/100 [00:53<01:20, 1.09s/trial, best loss: -0.6700077036377685] 28%|██▊ | 28/100 [00:56<01:30, 1.26s/trial, best loss: -0.6700077036377685] 29%|██▉ | 29/100 [00:57<01:25, 1.20s/trial, best loss: -0.6700077036377685] 30%|███ | 30/100 [01:00<01:54, 1.63s/trial, best loss: -0.6700077036377685] 31%|███ | 31/100 [01:01<01:41, 1.48s/trial, best loss: -0.6700077036377685] 32%|███▏ | 32/100 [01:03<01:50, 1.62s/trial, best loss: -0.6700077036377685] 33%|███▎ | 33/100 [01:05<01:55, 1.73s/trial, best loss: -0.6700077036377685] 34%|███▍ | 34/100 [01:06<01:40, 1.52s/trial, best loss: -0.6700077036377685] 35%|███▌ | 35/100 [01:07<01:29, 1.38s/trial, best loss: -0.6700077036377685] 37%|███▋ | 37/100 [01:08<01:01, 1.02trial/s, best loss: -0.6700077036377685] 38%|███▊ | 38/100 [01:09<01:01, 1.01trial/s, best loss: -0.6700077036377685] 40%|████ | 40/100 [01:10<00:47, 1.27trial/s, best loss: -0.6700077036377685] 41%|████ | 41/100 [01:14<01:29, 1.52s/trial, best loss: -0.6700077036377685] 43%|████▎ | 43/100 [01:15<01:04, 1.12s/trial, best loss: -0.6700077036377685] 45%|████▌ | 45/100 [01:16<00:49, 1.11trial/s, best loss: -0.6700077036377685] 47%|████▋ | 47/100 [01:17<00:40, 1.31trial/s, best loss: -0.6701226405919697] 48%|████▊ | 48/100 [01:18<00:42, 1.23trial/s, best loss: -0.6701226405919697] 49%|████▉ | 49/100 [01:19<00:43, 1.17trial/s, best loss: -0.6701226405919697] 51%|█████ | 51/100 [01:21<00:44, 1.10trial/s, best loss: -0.6701226405919697] 52%|█████▏ | 52/100 [01:24<01:05, 1.37s/trial, best loss: -0.6701226405919697] 53%|█████▎ | 53/100 [01:25<01:00, 1.28s/trial, best loss: -0.6701226405919697] 55%|█████▌ | 55/100 [01:27<00:52, 1.17s/trial, best loss: -0.6701226405919697] 56%|█████▌ | 56/100 [01:28<00:49, 1.14s/trial, best loss: -0.6701226405919697] 57%|█████▋ | 57/100 [01:33<01:29, 2.08s/trial, best loss: -0.6701226405919697] 58%|█████▊ | 58/100 [01:40<02:20, 3.34s/trial, best loss: -0.6701226405919697] 59%|█████▉ | 59/100 [01:43<02:13, 3.26s/trial, best loss: -0.6701226405919697] 60%|██████ | 60/100 [01:46<02:07, 3.19s/trial, best loss: -0.6701226405919697] 61%|██████ | 61/100 [01:49<02:02, 3.14s/trial, best loss: -0.6701226405919697] 62%|██████▏ | 62/100 [01:50<01:35, 2.52s/trial, best loss: -0.6701226405919697] 63%|██████▎ | 63/100 [01:58<02:32, 4.13s/trial, best loss: -0.6701226405919697] 64%|██████▍ | 64/100 [02:01<02:16, 3.80s/trial, best loss: -0.6701226405919697] 65%|██████▌ | 65/100 [02:02<01:44, 2.97s/trial, best loss: -0.6701226405919697] 66%|██████▌ | 66/100 [02:22<04:33, 8.04s/trial, best loss: -0.6701226405919697] 

In [0]:
modelo_ab = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_restrito, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_ab.predict_proba(X_train[restrito])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_ab.predict_proba(temp_test[restrito])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (26.58404123642025, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (24.588427806642237, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = restrito
modelos = [prefixo, *cols]

tab_KS = calcula_KS(modelo_ab, X_train, X_test, y_train, y_test, target, nscores=cols, restrito=restrito, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.534322,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.376142,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.753724,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.205043,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.854305,1.615136,NaN,2.530711,2.487262,2.958390
5,2.576500,NaN,NaN,2.321931,2.219712,2.280344
6,2.341115,NaN,NaN,1.840165,2.005944,2.096871
7,1.787363,NaN,NaN,1.717734,1.600620,1.815956
8,1.493035,NaN,NaN,1.309091,1.480186,1.231609


#### Aberto

In [0]:
prefixo = "ab_aberto"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="adaboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [01:20<2:12:21, 80.22s/trial, best loss: -0.6641221132762672] 2%|▏ | 2/100 [02:14<1:45:53, 64.83s/trial, best loss: -0.6641221132762672] 3%|▎ | 3/100 [02:35<1:12:28, 44.83s/trial, best loss: -0.6654811541030651] 4%|▍ | 4/100 [03:02<1:00:29, 37.80s/trial, best loss: -0.6654811541030651] 5%|▌ | 5/100 [03:17<46:50, 29.59s/trial, best loss: -0.6654811541030651]  6%|▌ | 6/100 [03:19<31:39, 20.21s/trial, best loss: -0.6654811541030651] 7%|▋ | 7/100 [03:37<30:13, 19.50s/trial, best loss: -0.6692360618703073] 8%|▊ | 8/100 [03:41<22:20, 14.57s/trial, best loss: -0.6692360618703073] 9%|▉ | 9/100 [03:43<16:08, 10.64s/trial, best loss: -0.6694698376125734] 10%|█ | 10/100 [03:53<15:40, 10.45s/trial, best loss: -0.6694698376125734] 11%|█ | 11/100 [03:54<11:12, 7.56s/trial, best loss: -0.6694698376125734] 12%|█▏ | 12/100 [03:56<08:36, 5.87s/trial, best loss: -0.6694698376125734] 13%|█▎ | 13/100 [03:57<06:22, 4.40s/trial, best loss: -0.6694698376125734] 14%|█▍ | 14/100 [04:05<07:52, 5.49s/trial, best loss: -0.6694698376125734] 15%|█▌ | 15/100 [04:07<06:17, 4.44s/trial, best loss: -0.6694698376125734] 16%|█▌ | 16/100 [04:08<04:46, 3.41s/trial, best loss: -0.6694698376125734] 17%|█▋ | 17/100 [04:09<03:42, 2.68s/trial, best loss: -0.6694698376125734] 18%|█▊ | 18/100 [04:10<02:58, 2.18s/trial, best loss: -0.6694698376125734] 19%|█▉ | 19/100 [04:11<02:28, 1.83s/trial, best loss: -0.6694698376125734] 20%|██ | 20/100 [04:12<02:06, 1.58s/trial, best loss: -0.6694698376125734] 21%|██ | 21/100 [04:15<02:38, 2.01s/trial, best loss: -0.6694698376125734] 22%|██▏ | 22/100 [04:16<02:13, 1.71s/trial, best loss: -0.6694698376125734] 24%|██▍ | 24/100 [04:20<02:20, 1.85s/trial, best loss: -0.669579932687899]  25%|██▌ | 25/100 [04:21<02:02, 1.64s/trial, best loss: -0.669579932687899] 27%|██▋ | 27/100 [04:24<01:55, 1.58s/trial, best loss: -0.669579932687899] 28%|██▊ | 28/100 [04:26<02:01, 1.68s/trial, best loss: -0.669579932687899] 29%|██▉ | 29/100 [04:28<02:05, 1.76s/trial, best loss: -0.669579932687899] 30%|███ | 30/100 [04:32<02:44, 2.35s/trial, best loss: -0.669579932687899] 31%|███ | 31/100 [04:43<05:24, 4.70s/trial, best loss: -0.669579932687899] 32%|███▏ | 32/100 [04:47<05:06, 4.50s/trial, best loss: -0.669579932687899] 34%|███▍ | 34/100 [04:51<03:44, 3.40s/trial, best loss: -0.669579932687899] 35%|███▌ | 35/100 [04:58<04:37, 4.27s/trial, best loss: -0.669579932687899] 36%|███▌ | 36/100 [05:04<05:01, 4.71s/trial, best loss: -0.669579932687899] 37%|███▋ | 37/100 [05:08<04:45, 4.53s/trial, best loss: -0.669579932687899] 38%|███▊ | 38/100 [05:10<03:57, 3.83s/trial, best loss: -0.669579932687899] 39%|███▉ | 39/100 [05:11<03:04, 3.03s/trial, best loss: -0.669579932687899] 40%|████ | 40/100 [05:13<02:44, 2.74s/trial, best loss: -0.669579932687899] 41%|████ | 41/100 [05:14<02:11, 2.23s/trial, best loss: -0.669579932687899] 42%|████▏ | 42/100 [05:27<05:13, 5.41s/trial, best loss: -0.669579932687899] 43%|████▎ | 43/100 [05:42<07:50, 8.25s/trial, best loss: -0.669579932687899] 44%|████▍ | 44/100 [05:47<06:48, 7.29s/trial, best loss: -0.669579932687899] 45%|████▌ | 45/100 [05:55<06:52, 7.51s/trial, best loss: -0.669579932687899] 46%|████▌ | 46/100 [05:59<05:48, 6.46s/trial, best loss: -0.669579932687899] 47%|████▋ | 47/100 [06:08<06:22, 7.23s/trial, best loss: -0.669579932687899] 48%|████▊ | 48/100 [06:11<05:10, 5.96s/trial, best loss: -0.669579932687899] 49%|████▉ | 49/100 [06:14<04:19, 5.08s/trial, best loss: -0.669579932687899] 50%|█████ | 50/100 [06:15<03:12, 3.86s/trial, best loss: -0.669579932687899] 51%|█████ | 51/100 [06:16<02:27, 3.00s/trial, best loss: -0.669579932687899] 52%|█████▏ | 52/100 [06:21<02:53, 3.61s/trial, best loss: -0.669579932687899] 53%|█████▎ | 53/100 [06:22<02:12, 2.83s/trial, best loss: -0.669579932687899] 55%|█████▌ | 55/100 [06:23<01:18, 1.75s/trial, best loss: -0.669579932687899] 56%|█████▌ | 56/100 [06:27<01:41, 2.32s/trial, be

In [0]:
modelo_ab_aberto = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_ab_aberto.predict_proba(X_train[aberto])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_ab_aberto.predict_proba(temp_test[aberto])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (26.706101913242343, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (24.374815138249094, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = aberto
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_ab_aberto, X_train, X_test, y_train, y_test, target, nscores=cols, aberto=aberto, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS
  
vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.534322,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.376142,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.753724,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.205043,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.854305,1.615136,NaN,2.530711,2.487262,2.958390
5,2.576500,NaN,NaN,2.321931,2.219712,2.280344
6,2.341115,NaN,NaN,1.840165,2.005944,2.096871
7,1.787363,NaN,NaN,1.717734,1.600620,1.815956
8,1.493035,NaN,NaN,1.309091,1.480186,1.231609


#### Aberto Contínuo

In [0]:
prefixo = "ab_aberto_continuo"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="adaboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto_continuo, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:28<46:26, 28.15s/trial, best loss: -0.5435062640331231] 2%|▏ | 2/100 [00:31<21:49, 13.37s/trial, best loss: -0.666225877700984]  3%|▎ | 3/100 [00:34<13:58, 8.64s/trial, best loss: -0.666225877700984] 4%|▍ | 4/100 [00:39<11:32, 7.21s/trial, best loss: -0.6680936218727657] 5%|▌ | 5/100 [00:51<14:09, 8.94s/trial, best loss: -0.6686383240340957] 6%|▌ | 6/100 [00:52<09:47, 6.25s/trial, best loss: -0.6686383240340957] 7%|▋ | 7/100 [00:53<07:01, 4.53s/trial, best loss: -0.6686383240340957] 8%|▊ | 8/100 [01:07<11:34, 7.55s/trial, best loss: -0.6686383240340957] 9%|▉ | 9/100 [01:16<12:08, 8.01s/trial, best loss: -0.6686383240340957] 10%|█ | 10/100 [01:21<10:37, 7.08s/trial, best loss: -0.6686383240340957] 11%|█ | 11/100 [01:26<09:34, 6.45s/trial, best loss: -0.6686383240340957] 12%|█▏ | 12/100 [01:27<07:01, 4.79s/trial, best loss: -0.6686383240340957] 13%|█▎ | 13/100 [01:33<07:29, 5.16s/trial, best loss: -0.6686383240340957] 14%|█▍ | 14/100 [01:42<09:04, 6.33s/trial, best loss: -0.6686383240340957] 15%|█▌ | 15/100 [02:09<17:48, 12.57s/trial, best loss: -0.6686383240340957] 16%|█▌ | 16/100 [02:14<14:24, 10.29s/trial, best loss: -0.6686383240340957] 17%|█▋ | 17/100 [02:16<10:47, 7.80s/trial, best loss: -0.6686383240340957] 18%|█▊ | 18/100 [02:17<07:52, 5.76s/trial, best loss: -0.6686383240340957] 20%|██ | 20/100 [02:18<04:26, 3.33s/trial, best loss: -0.6686383240340957] 21%|██ | 21/100 [02:41<10:49, 8.22s/trial, best loss: -0.6686383240340957] 22%|██▏ | 22/100 [02:43<08:34, 6.59s/trial, best loss: -0.6686383240340957] 23%|██▎ | 23/100 [03:09<15:15, 11.88s/trial, best loss: -0.6686383240340957] 24%|██▍ | 24/100 [03:26<16:52, 13.32s/trial, best loss: -0.6686383240340957] 25%|██▌ | 25/100 [03:37<15:49, 12.66s/trial, best loss: -0.6686383240340957] 26%|██▌ | 26/100 [04:49<36:51, 29.89s/trial, best loss: -0.6686383240340957] 27%|██▋ | 27/100 [05:25<38:33, 31.69s/trial, best loss: -0.6686383240340957] 28%|██▊ | 28/100 [05:37<31:03, 25.89s/trial, best loss: -0.6686383240340957] 29%|██▉ | 29/100 [05:39<22:15, 18.81s/trial, best loss: -0.6690337102810746] 31%|███ | 31/100 [06:02<17:47, 15.47s/trial, best loss: -0.6690337102810746] 32%|███▏ | 32/100 [06:10<15:26, 13.63s/trial, best loss: -0.6694302211223088] 33%|███▎ | 33/100 [07:15<30:11, 27.04s/trial, best loss: -0.6694302211223088] 34%|███▍ | 34/100 [07:46<30:56, 28.13s/trial, best loss: -0.6694302211223088] 35%|███▌ | 35/100 [08:50<41:21, 38.17s/trial, best loss: -0.6694302211223088] 36%|███▌ | 36/100 [08:55<30:37, 28.71s/trial, best loss: -0.6694302211223088] 37%|███▋ | 37/100 [11:04<1:00:22, 57.50s/trial, best loss: -0.6695036138522873] 38%|███▊ | 38/100 [11:06<42:38, 41.27s/trial, best loss: -0.6695036138522873]  39%|███▉ | 39/100 [11:10<30:47, 30.28s/trial, best loss: -0.6695036138522873] 40%|████ | 40/100 [11:11<21:36, 21.61s/trial, best loss: -0.6695036138522873] 41%|████ | 41/100 [11:19<17:16, 17.57s/trial, best loss: -0.6695036138522873] 43%|████▎ | 43/100 [11:26<10:33, 11.11s/trial, best loss: -0.6695036138522873] 44%|████▍ | 44/100 [11:48<12:53, 13.81s/trial, best loss: -0.6695036138522873] 45%|████▌ | 45/100 [12:17<16:18, 17.78s/trial, best loss: -0.6695036138522873] 46%|████▌ | 46/100 [12:40<17:17, 19.21s/trial, best loss: -0.6695036138522873] 47%|████▋ | 47/100 [12:45<13:27, 15.24s/trial, best loss: -0.6695036138522873] 48%|████▊ | 48/100 [13:02<13:39, 15.75s/trial, best loss: -0.6695036138522873] 49%|████▉ | 49/100 [13:10<11:29, 13.51s/trial, best loss: -0.6695036138522873] 50%|█████ | 50/100 [14:11<22:51, 27.43s/trial, best loss: -0.6695036138522873] 51%|█████ | 51/100 [14:26<19:24, 23.77s/trial, best loss: -0.6695036138522873] 52%|█████▏ | 52/100 [14:40<16:42, 20.89s/trial, best loss: -0.6695036138522873] 53%|█████▎ | 53/100 [14:43<12:11, 15.57s/trial, best loss: -0.6695036138522873] 54%|█████▍ | 54/100 [15:00<12:16, 16.00s/trial, best loss: -0.6695036138522873] 55%|

In [0]:
modelo_ab_continuo = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto_continuo, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_ab_continuo.predict_proba(X_train[aberto_continuo])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_ab_continuo.predict_proba(temp_test[aberto_continuo])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (27.598206396318748, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (24.09523823293481, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = aberto_continuo
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_ab_continuo, X_train, X_test, y_train, y_test, target, nscores=cols, continuo=aberto_continuo, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.534322,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.376142,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.753724,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.205043,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.854305,1.615136,NaN,2.530711,2.487262,2.958390
5,2.576500,NaN,NaN,2.321931,2.219712,2.280344
6,2.341115,NaN,NaN,1.840165,2.005944,2.096871
7,1.787363,NaN,NaN,1.717734,1.600620,1.815956
8,1.493035,NaN,NaN,1.309091,1.480186,1.231609


### XgBoost

#### Restrito

In [0]:
prefixo = "xg_restrito"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="xgboost",  n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_restrito, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:49<1:21:43, 49.53s/trial, best loss: -0.6613160978105578] 2%|▏ | 2/100 [00:51<35:14, 21.58s/trial, best loss: -0.6638105895820932]  3%|▎ | 3/100 [01:00<25:37, 15.85s/trial, best loss: -0.6680389865464239] 4%|▍ | 4/100 [01:02<16:37, 10.39s/trial, best loss: -0.6680389865464239] 5%|▌ | 5/100 [01:13<16:48, 10.62s/trial, best loss: -0.6680389865464239] 6%|▌ | 6/100 [01:24<16:50, 10.75s/trial, best loss: -0.6680389865464239] 7%|▋ | 7/100 [01:29<13:45, 8.88s/trial, best loss: -0.6683598480561821] 8%|▊ | 8/100 [01:31<10:15, 6.69s/trial, best loss: -0.6683598480561821] 9%|▉ | 9/100 [01:33<07:55, 5.23s/trial, best loss: -0.6683598480561821] 10%|█ | 10/100 [01:42<09:36, 6.40s/trial, best loss: -0.6683598480561821] 11%|█ | 11/100 [01:47<08:52, 5.98s/trial, best loss: -0.6683598480561821] 12%|█▏ | 12/100 [01:49<06:59, 4.77s/trial, best loss: -0.6683598480561821] 13%|█▎ | 13/100 [01:50<05:15, 3.63s/trial, best loss: -0.6683598480561821] 14%|█▍ | 14/100 [01:56<06:14, 4.36s/trial, best loss: -0.6683598480561821] 15%|█▌ | 15/100 [02:03<07:18, 5.16s/trial, best loss: -0.6683598480561821] 16%|█▌ | 16/100 [02:07<06:26, 4.60s/trial, best loss: -0.6683598480561821] 18%|█▊ | 18/100 [02:12<04:57, 3.63s/trial, best loss: -0.6683598480561821] 19%|█▉ | 19/100 [02:16<05:02, 3.73s/trial, best loss: -0.6683598480561821] 20%|██ | 20/100 [02:17<04:01, 3.02s/trial, best loss: -0.6683598480561821] 21%|██ | 21/100 [02:27<06:29, 4.93s/trial, best loss: -0.6683598480561821] 22%|██▏ | 22/100 [02:28<04:58, 3.83s/trial, best loss: -0.6686968983182932] 23%|██▎ | 23/100 [02:29<03:53, 3.03s/trial, best loss: -0.6686968983182932] 24%|██▍ | 24/100 [02:35<04:56, 3.90s/trial, best loss: -0.6686968983182932] 25%|██▌ | 25/100 [02:45<07:06, 5.69s/trial, best loss: -0.6686968983182932] 26%|██▌ | 26/100 [02:47<05:40, 4.61s/trial, best loss: -0.6686968983182932] 27%|██▋ | 27/100 [02:49<04:40, 3.84s/trial, best loss: -0.6686968983182932] 28%|██▊ | 28/100 [02:51<03:57, 3.30s/trial, best loss: -0.6686968983182932] 29%|██▉ | 29/100 [02:56<04:30, 3.81s/trial, best loss: -0.6686968983182932] 30%|███ | 30/100 [02:59<04:10, 3.57s/trial, best loss: -0.6689515151905482] 31%|███ | 31/100 [03:00<03:13, 2.81s/trial, best loss: -0.6689515151905482] 32%|███▏ | 32/100 [03:01<02:34, 2.27s/trial, best loss: -0.6689515151905482] 33%|███▎ | 33/100 [03:04<02:47, 2.50s/trial, best loss: -0.6689515151905482] 35%|███▌ | 35/100 [03:17<04:42, 4.35s/trial, best loss: -0.6689515151905482] 36%|███▌ | 36/100 [03:21<04:33, 4.27s/trial, best loss: -0.6689515151905482] 37%|███▋ | 37/100 [03:23<03:52, 3.68s/trial, best loss: -0.6689515151905482] 38%|███▊ | 38/100 [03:34<05:52, 5.68s/trial, best loss: -0.6689515151905482] 39%|███▉ | 39/100 [03:37<05:01, 4.94s/trial, best loss: -0.6689515151905482] 40%|████ | 40/100 [03:46<06:06, 6.11s/trial, best loss: -0.6689515151905482] 41%|████ | 41/100 [03:55<06:50, 6.96s/trial, best loss: -0.6689515151905482] 43%|████▎ | 43/100 [04:00<04:41, 4.94s/trial, best loss: -0.6689515151905482] 44%|████▍ | 44/100 [04:05<04:38, 4.97s/trial, best loss: -0.6689515151905482] 45%|████▌ | 45/100 [04:07<03:51, 4.20s/trial, best loss: -0.6689515151905482] 47%|████▋ | 47/100 [04:09<02:30, 2.85s/trial, best loss: -0.6689515151905482] 48%|████▊ | 48/100 [04:11<02:17, 2.65s/trial, best loss: -0.6689515151905482] 49%|████▉ | 49/100 [04:29<05:31, 6.50s/trial, best loss: -0.6689515151905482] 50%|█████ | 50/100 [04:35<05:18, 6.37s/trial, best loss: -0.6689515151905482] 51%|█████ | 51/100 [04:38<04:27, 5.46s/trial, best loss: -0.6689515151905482] 52%|█████▏ | 52/100 [04:43<04:16, 5.34s/trial, best loss: -0.6689515151905482] 53%|█████▎ | 53/100 [04:47<03:53, 4.96s/trial, best loss: -0.6689515151905482] 54%|█████▍ | 54/100 [04:48<02:55, 3.82s/trial, best loss: -0.6689515151905482] 55%|█████▌ | 55/100 [04:51<02:41, 3.58s/trial, best loss: -0.6689515151905482] 56%|█████▌ | 56/100 [04:54<02:3

In [0]:
modelo_xg = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_restrito, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_xg.predict_proba(X_train[restrito])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_xg.predict_proba(temp_test[restrito])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (29.818994121062627, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (24.831723505398077, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = restrito
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_xg, X_train, X_test, y_train, y_test, target, nscores=cols, restrito=restrito, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.534322,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.376142,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.753724,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.205043,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.854305,1.615136,NaN,2.530711,2.487262,2.958390
5,2.576500,NaN,NaN,2.321931,2.219712,2.280344
6,2.341115,NaN,NaN,1.840165,2.005944,2.096871
7,1.787363,NaN,NaN,1.717734,1.600620,1.815956
8,1.493035,NaN,NaN,1.309091,1.480186,1.231609


#### Aberto

In [0]:
prefixo = "xg_aberto"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="xgboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [01:15<2:04:22, 75.38s/trial, best loss: -0.6558137183268374] 2%|▏ | 2/100 [01:18<53:35, 32.81s/trial, best loss: -0.6611237824426212]  3%|▎ | 3/100 [01:19<29:34, 18.29s/trial, best loss: -0.6631658215875063] 4%|▍ | 4/100 [01:29<24:02, 15.03s/trial, best loss: -0.6634605050741771] 5%|▌ | 5/100 [01:51<27:47, 17.56s/trial, best loss: -0.6756156512419621] 6%|▌ | 6/100 [02:03<24:33, 15.68s/trial, best loss: -0.6756156512419621] 7%|▋ | 7/100 [02:08<18:53, 12.19s/trial, best loss: -0.6756156512419621] 8%|▊ | 8/100 [02:37<26:55, 17.55s/trial, best loss: -0.6756156512419621] 9%|▉ | 9/100 [02:40<19:43, 13.01s/trial, best loss: -0.6756156512419621] 10%|█ | 10/100 [02:45<15:48, 10.54s/trial, best loss: -0.6756156512419621] 11%|█ | 11/100 [02:49<12:40, 8.54s/trial, best loss: -0.6756156512419621] 12%|█▏ | 12/100 [03:04<15:25, 10.52s/trial, best loss: -0.6756156512419621] 13%|█▎ | 13/100 [03:53<32:10, 22.20s/trial, best loss: -0.6756156512419621] 14%|█▍ | 14/100 [04:12<30:26, 21.24s/trial, best loss: -0.6756156512419621] 15%|█▌ | 15/100 [04:34<30:25, 21.48s/trial, best loss: -0.6756156512419621] 16%|█▌ | 16/100 [04:49<27:21, 19.54s/trial, best loss: -0.6756156512419621] 17%|█▋ | 17/100 [04:56<21:49, 15.77s/trial, best loss: -0.6756156512419621] 18%|█▊ | 18/100 [05:04<18:22, 13.44s/trial, best loss: -0.6756156512419621] 19%|█▉ | 19/100 [05:15<17:10, 12.72s/trial, best loss: -0.6756156512419621] 20%|██ | 20/100 [05:19<13:28, 10.11s/trial, best loss: -0.6756156512419621] 21%|██ | 21/100 [05:27<12:29, 9.48s/trial, best loss: -0.6756156512419621] 22%|██▏ | 22/100 [05:28<09:01, 6.94s/trial, best loss: -0.6756156512419621] 23%|██▎ | 23/100 [05:32<07:46, 6.06s/trial, best loss: -0.6756156512419621] 25%|██▌ | 25/100 [05:45<07:50, 6.27s/trial, best loss: -0.6756156512419621] 26%|██▌ | 26/100 [06:02<10:43, 8.70s/trial, best loss: -0.6756156512419621] 29%|██▉ | 29/100 [06:08<06:12, 5.25s/trial, best loss: -0.6756156512419621] 30%|███ | 30/100 [06:15<06:32, 5.60s/trial, best loss: -0.6756156512419621] 31%|███ | 31/100 [06:21<06:33, 5.70s/trial, best loss: -0.6756156512419621] 32%|███▏ | 32/100 [06:27<06:32, 5.77s/trial, best loss: -0.6756156512419621] 33%|███▎ | 33/100 [06:34<06:48, 6.09s/trial, best loss: -0.6756156512419621] 34%|███▍ | 34/100 [06:39<06:23, 5.81s/trial, best loss: -0.6756156512419621] 35%|███▌ | 35/100 [06:42<05:27, 5.04s/trial, best loss: -0.6756156512419621] 36%|███▌ | 36/100 [07:06<11:06, 10.41s/trial, best loss: -0.6756156512419621] 37%|███▋ | 37/100 [07:07<08:05, 7.71s/trial, best loss: -0.6756156512419621] 39%|███▉ | 39/100 [07:11<05:13, 5.14s/trial, best loss: -0.6756156512419621] 40%|████ | 40/100 [07:15<04:52, 4.87s/trial, best loss: -0.6756156512419621] 41%|████ | 41/100 [07:18<04:19, 4.39s/trial, best loss: -0.6756156512419621] 42%|████▏ | 42/100 [07:20<03:37, 3.75s/trial, best loss: -0.6756156512419621] 43%|████▎ | 43/100 [07:21<02:50, 2.99s/trial, best loss: -0.6756156512419621] 44%|████▍ | 44/100 [07:22<02:15, 2.43s/trial, best loss: -0.6756156512419621] 45%|████▌ | 45/100 [07:32<04:14, 4.63s/trial, best loss: -0.6756156512419621] 47%|████▋ | 47/100 [07:37<03:14, 3.67s/trial, best loss: -0.6756156512419621] 48%|████▊ | 48/100 [07:39<02:49, 3.27s/trial, best loss: -0.6756156512419621] 49%|████▉ | 49/100 [07:42<02:43, 3.21s/trial, best loss: -0.6756156512419621] 50%|█████ | 50/100 [07:43<02:11, 2.62s/trial, best loss: -0.6756156512419621] 51%|█████ | 51/100 [07:48<02:41, 3.29s/trial, best loss: -0.6756156512419621] 52%|█████▏ | 52/100 [07:50<02:20, 2.93s/trial, best loss: -0.6756156512419621] 53%|█████▎ | 53/100 [07:58<03:26, 4.40s/trial, best loss: -0.6756156512419621] 54%|█████▍ | 54/100 [08:02<03:17, 4.29s/trial, best loss: -0.6756156512419621] 55%|█████▌ | 55/100 [08:41<10:53, 14.53s/trial, best loss: -0.6756156512419621] 56%|█████▌ | 56/100 [08:47<08:48, 12.01s/trial, best loss: -0.6756156512419621] 57%|█████▋ 

In [0]:
modelo_xg_aberto = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_xg_aberto.predict_proba(X_train[aberto])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_xg_aberto.predict_proba(temp_test[aberto])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (38.91193445596118, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (23.792357410498866, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = aberto
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_xg_aberto, X_train, X_test, y_train, y_test, target, nscores=cols, aberto=aberto, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS
  
vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.534322,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.376142,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.753724,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.205043,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.854305,1.615136,NaN,2.530711,2.487262,2.958390
5,2.576500,NaN,NaN,2.321931,2.219712,2.280344
6,2.341115,NaN,NaN,1.840165,2.005944,2.096871
7,1.787363,NaN,NaN,1.717734,1.600620,1.815956
8,1.493035,NaN,NaN,1.309091,1.480186,1.231609


#### Aberto Contínuo

In [0]:
prefixo = "xg_aberto_continuo"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="xgboost", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto_continuo, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:37<1:01:26, 37.24s/trial, best loss: -0.6605837703529748] 2%|▏ | 2/100 [00:48<35:38, 21.82s/trial, best loss: -0.6640152381943797]  3%|▎ | 3/100 [00:52<22:07, 13.69s/trial, best loss: -0.6640152381943797] 4%|▍ | 4/100 [00:56<15:47, 9.87s/trial, best loss: -0.6640152381943797] 5%|▌ | 5/100 [01:00<12:17, 7.76s/trial, best loss: -0.6640152381943797] 6%|▌ | 6/100 [01:01<08:33, 5.47s/trial, best loss: -0.6640152381943797] 7%|▋ | 7/100 [01:04<07:13, 4.66s/trial, best loss: -0.668425194535571]  8%|▊ | 8/100 [01:06<05:51, 3.82s/trial, best loss: -0.668425194535571] 9%|▉ | 9/100 [01:10<05:53, 3.88s/trial, best loss: -0.668425194535571] 10%|█ | 10/100 [01:16<06:48, 4.54s/trial, best loss: -0.668425194535571] 12%|█▏ | 12/100 [01:23<05:57, 4.06s/trial, best loss: -0.668425194535571] 13%|█▎ | 13/100 [01:27<05:52, 4.05s/trial, best loss: -0.668425194535571] 14%|█▍ | 14/100 [01:29<05:02, 3.51s/trial, best loss: -0.668425194535571] 15%|█▌ | 15/100 [01:37<06:43, 4.75s/trial, best loss: -0.668425194535571] 17%|█▋ | 17/100 [01:51<07:56, 5.74s/trial, best loss: -0.668425194535571] 18%|█▊ | 18/100 [01:52<06:18, 4.61s/trial, best loss: -0.668425194535571] 19%|█▉ | 19/100 [01:55<05:41, 4.21s/trial, best loss: -0.668425194535571] 20%|██ | 20/100 [01:58<05:11, 3.89s/trial, best loss: -0.668425194535571] 21%|██ | 21/100 [02:05<06:15, 4.76s/trial, best loss: -0.668425194535571] 22%|██▏ | 22/100 [02:11<06:38, 5.11s/trial, best loss: -0.668425194535571] 25%|██▌ | 25/100 [02:13<03:21, 2.68s/trial, best loss: -0.668425194535571] 26%|██▌ | 26/100 [02:14<02:52, 2.34s/trial, best loss: -0.668425194535571] 27%|██▋ | 27/100 [02:21<04:08, 3.41s/trial, best loss: -0.668425194535571] 28%|██▊ | 28/100 [02:22<03:23, 2.82s/trial, best loss: -0.668425194535571] 29%|██▉ | 29/100 [02:24<03:05, 2.61s/trial, best loss: -0.668425194535571] 30%|███ | 30/100 [02:26<02:51, 2.45s/trial, best loss: -0.668425194535571] 31%|███ | 31/100 [02:29<03:00, 2.61s/trial, best loss: -0.668425194535571] 32%|███▏ | 32/100 [02:36<04:22, 3.87s/trial, best loss: -0.668425194535571] 33%|███▎ | 33/100 [02:39<04:02, 3.62s/trial, best loss: -0.668425194535571] 36%|███▌ | 36/100 [02:46<03:06, 2.91s/trial, best loss: -0.668425194535571] 38%|███▊ | 38/100 [02:55<03:35, 3.47s/trial, best loss: -0.668425194535571] 39%|███▉ | 39/100 [03:00<03:50, 3.78s/trial, best loss: -0.668425194535571] 41%|████ | 41/100 [03:08<03:47, 3.86s/trial, best loss: -0.668425194535571] 42%|████▏ | 42/100 [03:09<03:10, 3.28s/trial, best loss: -0.668425194535571] 43%|████▎ | 43/100 [03:12<03:03, 3.23s/trial, best loss: -0.668425194535571] 44%|████▍ | 44/100 [03:15<02:44, 2.93s/trial, best loss: -0.668425194535571] 45%|████▌ | 45/100 [03:20<03:10, 3.47s/trial, best loss: -0.668425194535571] 46%|████▌ | 46/100 [03:25<03:30, 3.89s/trial, best loss: -0.668425194535571] 47%|████▋ | 47/100 [03:30<03:42, 4.21s/trial, best loss: -0.668425194535571] 48%|████▊ | 48/100 [03:35<03:50, 4.44s/trial, best loss: -0.668425194535571] 49%|████▉ | 49/100 [03:38<03:25, 4.03s/trial, best loss: -0.668425194535571] 51%|█████ | 51/100 [03:39<01:59, 2.43s/trial, best loss: -0.668425194535571] 52%|█████▏ | 52/100 [03:42<02:04, 2.58s/trial, best loss: -0.6689004954283971] 53%|█████▎ | 53/100 [03:43<01:42, 2.18s/trial, best loss: -0.6689004954283971] 54%|█████▍ | 54/100 [03:45<01:38, 2.14s/trial, best loss: -0.6690272452539902] 55%|█████▌ | 55/100 [03:46<01:21, 1.82s/trial, best loss: -0.6690272452539902] 56%|█████▌ | 56/100 [03:48<01:23, 1.89s/trial, best loss: -0.6690272452539902] 57%|█████▋ | 57/100 [03:49<01:10, 1.64s/trial, best loss: -0.6690272452539902] 58%|█████▊ | 58/100 [03:55<02:02, 2.92s/trial, best loss: -0.6690272452539902] 59%|█████▉ | 59/100 [03:56<01:36, 2.36s/trial, best loss: -0.6690272452539902] 60%|██████ | 60/100 [03:57<01:18, 1.96s/trial, best loss: -0.6690272452539902] 61%|██████ | 61/100 [03:59<01:17, 1.98s/trial, best l

In [0]:
modelo_xg_continuo = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto_continuo, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_xg_continuo.predict_proba(X_train[aberto_continuo])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_xg_continuo.predict_proba(temp_test[aberto_continuo])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (41.95175415689758, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (21.923189605690617, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = aberto_continuo
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_xg_continuo, X_train, X_test, y_train, y_test, target, nscores=cols, continuo=aberto_continuo, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.534322,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.376142,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.753724,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.205043,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.854305,1.615136,NaN,2.530711,2.487262,2.958390
5,2.576500,NaN,NaN,2.321931,2.219712,2.280344
6,2.341115,NaN,NaN,1.840165,2.005944,2.096871
7,1.787363,NaN,NaN,1.717734,1.600620,1.815956
8,1.493035,NaN,NaN,1.309091,1.480186,1.231609


### LightGBM

#### Restrito

In [0]:
prefixo = "lgbm_restrito"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="light_gbm",  n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_restrito, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [01:41<2:47:14, 101.36s/trial, best loss: -0.6611261921351229] 2%|▏ | 2/100 [02:02<1:28:23, 54.11s/trial, best loss: -0.6688648976982573]  3%|▎ | 3/100 [02:52<1:24:29, 52.27s/trial, best loss: -0.6688648976982573] 4%|▍ | 4/100 [03:18<1:07:03, 41.92s/trial, best loss: -0.6688648976982573] 5%|▌ | 5/100 [05:31<1:58:27, 74.82s/trial, best loss: -0.6688648976982573] 7%|▋ | 7/100 [05:43<1:01:50, 39.90s/trial, best loss: -0.6688648976982573] 8%|▊ | 8/100 [06:10<55:56, 36.48s/trial, best loss: -0.6688648976982573]  9%|▉ | 9/100 [06:13<41:21, 27.27s/trial, best loss: -0.6688648976982573] 10%|█ | 10/100 [06:57<48:00, 32.01s/trial, best loss: -0.6688648976982573] 11%|█ | 11/100 [07:04<36:49, 24.83s/trial, best loss: -0.6688648976982573] 12%|█▏ | 12/100 [07:32<37:47, 25.76s/trial, best loss: -0.6688648976982573] 13%|█▎ | 13/100 [07:44<31:30, 21.73s/trial, best loss: -0.6688648976982573] 14%|█▍ | 14/100 [07:58<27:53, 19.45s/trial, best loss: -0.6688648976982573] 16%|█▌ | 16/100 [08:09<18:18, 13.07s/trial, best loss: -0.6688648976982573] 17%|█▋ | 17/100 [08:31<20:48, 15.04s/trial, best loss: -0.6688648976982573] 18%|█▊ | 18/100 [09:08<28:23, 20.77s/trial, best loss: -0.6688648976982573] 19%|█▉ | 19/100 [09:57<38:24, 28.45s/trial, best loss: -0.6688648976982573]

In [0]:
modelo_lgbm = otimizador.modelo
target = 'inad30'

X_train, X_test, y_train, y_test = train_test_split(X_restrito, y, train_size=.7, random_state=random_state)
 
temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_lgbm.predict_proba(X_train[restrito])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_lgbm.predict_proba(temp_test[restrito])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (29.835997887997266, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (25.617835299620506, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = restrito
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_lgbm, X_train, X_test, y_train, y_test, target, nscores=cols, restrito=restrito, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.602261,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.397801,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.839268,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.182059,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.792703,1.615136,NaN,2.530711,2.487262,2.958390
5,2.472163,NaN,NaN,2.321931,2.219712,2.280344
6,2.330971,NaN,NaN,1.840165,2.005944,2.096871
7,1.805078,NaN,NaN,1.717734,1.600620,1.815956
8,1.493231,NaN,NaN,1.309091,1.480186,1.231609


#### Aberto

In [0]:
prefixo = "lgbm_aberto"
prefixos.append(prefixo)


otimizador = OtimizadorBayesiano(classificador="light_gbm", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [01:37<2:40:45, 97.42s/trial, best loss: -0.6645513947760232] 2%|▏ | 2/100 [02:34<2:00:21, 73.69s/trial, best loss: -0.6645513947760232] 3%|▎ | 3/100 [02:40<1:09:10, 42.79s/trial, best loss: -0.6645513947760232] 4%|▍ | 4/100 [02:55<50:55, 31.83s/trial, best loss: -0.6645513947760232]  5%|▌ | 5/100 [03:41<58:31, 36.97s/trial, best loss: -0.6647520126299432] 6%|▌ | 6/100 [03:42<38:45, 24.74s/trial, best loss: -0.6647520126299432] 7%|▋ | 7/100 [04:07<38:30, 24.84s/trial, best loss: -0.6647520126299432] 8%|▊ | 8/100 [04:16<30:22, 19.81s/trial, best loss: -0.6647520126299432] 9%|▉ | 9/100 [04:18<21:36, 14.25s/trial, best loss: -0.6647520126299432] 10%|█ | 10/100 [04:23<17:05, 11.40s/trial, best loss: -0.6647520126299432] 11%|█ | 11/100 [04:29<14:28, 9.75s/trial, best loss: -0.6647520126299432] 12%|█▏ | 12/100 [05:05<26:02, 17.76s/trial, best loss: -0.6648348403331177] 13%|█▎ | 13/100 [05:25<26:45, 18.45s/trial, best loss: -0.6687375564026866] 14%|█▍ | 14/100 [06:28<45:46, 31.93s/trial, best loss: -0.6687375564026866] 15%|█▌ | 15/100 [06:41<36:44, 25.93s/trial, best loss: -0.6687375564026866] 16%|█▌ | 16/100 [06:54<30:52, 22.05s/trial, best loss: -0.6687375564026866] 17%|█▋ | 17/100 [06:59<23:25, 16.93s/trial, best loss: -0.6690339704866997] 18%|█▊ | 18/100 [07:03<17:50, 13.05s/trial, best loss: -0.6690339704866997] 19%|█▉ | 19/100 [07:28<22:29, 16.65s/trial, best loss: -0.6690339704866997] 20%|██ | 20/100 [07:42<21:09, 15.87s/trial, best loss: -0.6690339704866997] 21%|██ | 21/100 [07:44<15:25, 11.71s/trial, best loss: -0.6690339704866997] 22%|██▏ | 22/100 [07:56<15:21, 11.81s/trial, best loss: -0.6690339704866997] 23%|██▎ | 23/100 [08:40<27:34, 21.49s/trial, best loss: -0.6690339704866997] 24%|██▍ | 24/100 [08:44<20:34, 16.25s/trial, best loss: -0.6690339704866997] 25%|██▌ | 25/100 [09:06<22:28, 17.99s/trial, best loss: -0.6690339704866997] 26%|██▌ | 26/100 [09:22<21:27, 17.40s/trial, best loss: -0.6690339704866997] 27%|██▋ | 27/100 [10:08<31:38, 26.00s/trial, best loss: -0.6690339704866997] 28%|██▊ | 28/100 [10:11<22:55, 19.11s/trial, best loss: -0.6690339704866997] 29%|██▉ | 29/100 [10:35<24:21, 20.59s/trial, best loss: -0.6690339704866997] 30%|███ | 30/100 [11:54<44:30, 38.15s/trial, best loss: -0.6690339704866997] 31%|███ | 31/100 [12:44<47:59, 41.73s/trial, best loss: -0.6690339704866997] 34%|███▍ | 34/100 [12:57<22:46, 20.70s/trial, best loss: -0.6690339704866997] 35%|███▌ | 35/100 [12:58<17:55, 16.54s/trial, best loss: -0.6690339704866997] 36%|███▌ | 36/100 [13:14<17:31, 16.43s/trial, best loss: -0.6690339704866997] 37%|███▋ | 37/100 [13:43<20:25, 19.46s/trial, best loss: -0.6690339704866997] 38%|███▊ | 38/100 [14:34<28:44, 27.82s/trial, best loss: -0.6692842499124515] 39%|███▉ | 39/100 [14:48<24:28, 24.07s/trial, best loss: -0.6692842499124515] 40%|████ | 40/100 [15:01<20:43, 20.73s/trial, best loss: -0.6692842499124515] 41%|████ | 41/100 [15:11<17:25, 17.71s/trial, best loss: -0.6692842499124515] 42%|████▏ | 42/100 [15:45<21:44, 22.50s/trial, best loss: -0.6692842499124515] 43%|████▎ | 43/100 [16:02<19:53, 20.94s/trial, best loss: -0.6692842499124515] 44%|████▍ | 44/100 [16:35<22:56, 24.58s/trial, best loss: -0.6692842499124515] 45%|████▌ | 45/100 [16:42<17:37, 19.22s/trial, best loss: -0.6692842499124515] 46%|████▌ | 46/100 [17:16<21:19, 23.70s/trial, best loss: -0.6692842499124515] 48%|████▊ | 48/100 [17:17<11:19, 13.07s/trial, best loss: -0.6692842499124515] 49%|████▉ | 49/100 [17:27<10:29, 12.34s/trial, best loss: -0.6692842499124515] 50%|█████ | 50/100 [17:43<10:53, 13.08s/trial, best loss: -0.6692842499124515] 51%|█████ | 51/100 [17:48<08:54, 10.91s/trial, best loss: -0.6692842499124515] 52%|█████▏ | 52/100 [18:06<10:19, 12.91s/trial, best loss: -0.6692842499124515] 53%|█████▎ | 53/100 [18:18<09:55, 12.68s/trial, best loss: -0.6692842499124515] 54%|█████▍ | 54/100 [18:22<07:48, 10.18s/trial, best loss: -0.67039

In [0]:
modelo_lgbm_aberto = otimizador.modelo
target = 'inad30'

X_train, X_test, y_train, y_test = train_test_split(X_aberto, y, train_size=.7, random_state=random_state)
 
temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_lgbm_aberto.predict_proba(X_train[aberto])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_lgbm_aberto.predict_proba(temp_test[aberto])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (39.11667498915603, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (25.51313635104821, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = aberto
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_lgbm_aberto, X_train, X_test, y_train, y_test, target, nscores=cols, aberto=aberto, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.602261,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.397801,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.839268,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.182059,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.792703,1.615136,NaN,2.530711,2.487262,2.958390
5,2.472163,NaN,NaN,2.321931,2.219712,2.280344
6,2.330971,NaN,NaN,1.840165,2.005944,2.096871
7,1.805078,NaN,NaN,1.717734,1.600620,1.815956
8,1.493231,NaN,NaN,1.309091,1.480186,1.231609


#### Aberto Contínuo

In [0]:
prefixo = "lgbm_aberto_continuo"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="light_gbm", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto_continuo, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [01:32<2:32:44, 92.57s/trial, best loss: -0.6672650809400377] 2%|▏ | 2/100 [01:43<1:12:51, 44.61s/trial, best loss: -0.6672650809400377] 3%|▎ | 3/100 [01:55<48:03, 29.73s/trial, best loss: -0.6672650809400377]  4%|▍ | 4/100 [02:20<44:36, 27.88s/trial, best loss: -0.6672650809400377] 5%|▌ | 5/100 [02:43<41:23, 26.14s/trial, best loss: -0.6672650809400377] 6%|▌ | 6/100 [02:45<28:06, 17.94s/trial, best loss: -0.6672650809400377] 7%|▋ | 7/100 [02:50<21:15, 13.72s/trial, best loss: -0.6672650809400377] 8%|▊ | 8/100 [02:58<18:15, 11.91s/trial, best loss: -0.6672650809400377] 9%|▉ | 9/100 [03:14<20:00, 13.20s/trial, best loss: -0.6672650809400377] 10%|█ | 10/100 [03:17<15:05, 10.06s/trial, best loss: -0.6672650809400377] 11%|█ | 11/100 [04:29<43:05, 29.05s/trial, best loss: -0.6672650809400377] 12%|█▏ | 12/100 [04:40<34:33, 23.57s/trial, best loss: -0.6672650809400377] 13%|█▎ | 13/100 [04:59<31:44, 21.89s/trial, best loss: -0.6672650809400377] 14%|█▍ | 14/100 [05:35<37:31, 26.17s/trial, best loss: -0.6672650809400377] 15%|█▌ | 15/100 [05:43<29:19, 20.70s/trial, best loss: -0.6676189771433656] 16%|█▌ | 16/100 [05:52<24:04, 17.19s/trial, best loss: -0.6676189771433656] 17%|█▋ | 17/100 [05:55<17:53, 12.93s/trial, best loss: -0.6676189771433656] 18%|█▊ | 18/100 [05:56<12:46, 9.35s/trial, best loss: -0.6676189771433656] 19%|█▉ | 19/100 [06:12<15:20, 11.36s/trial, best loss: -0.6676189771433656] 20%|██ | 20/100 [06:16<12:12, 9.16s/trial, best loss: -0.6676189771433656] 21%|██ | 21/100 [07:02<26:39, 20.24s/trial, best loss: -0.6676189771433656] 22%|██▏ | 22/100 [07:13<22:43, 17.48s/trial, best loss: -0.6676189771433656] 23%|██▎ | 23/100 [07:18<17:38, 13.74s/trial, best loss: -0.6676189771433656] 24%|██▍ | 24/100 [07:33<17:53, 14.13s/trial, best loss: -0.6676189771433656] 26%|██▌ | 26/100 [07:37<10:31, 8.54s/trial, best loss: -0.6676189771433656] 27%|██▋ | 27/100 [07:40<08:43, 7.18s/trial, best loss: -0.6676189771433656] 28%|██▊ | 28/100 [07:43<07:18, 6.09s/trial, best loss: -0.6676189771433656] 29%|██▉ | 29/100 [07:45<05:54, 4.99s/trial, best loss: -0.6676189771433656] 30%|███ | 30/100 [08:04<10:25, 8.93s/trial, best loss: -0.6676189771433656] 31%|███ | 31/100 [08:32<16:19, 14.19s/trial, best loss: -0.6676189771433656] 32%|███▏ | 32/100 [08:38<13:24, 11.83s/trial, best loss: -0.6676189771433656] 33%|███▎ | 33/100 [08:57<15:33, 13.94s/trial, best loss: -0.6676189771433656] 34%|███▍ | 34/100 [09:27<20:33, 18.69s/trial, best loss: -0.6676189771433656] 35%|███▌ | 35/100 [09:31<15:32, 14.34s/trial, best loss: -0.6676189771433656] 36%|███▌ | 36/100 [09:40<13:36, 12.76s/trial, best loss: -0.6676189771433656] 37%|███▋ | 37/100 [09:42<10:02, 9.56s/trial, best loss: -0.6676189771433656] 38%|███▊ | 38/100 [09:45<07:51, 7.61s/trial, best loss: -0.6676189771433656] 39%|███▉ | 39/100 [09:48<06:20, 6.24s/trial, best loss: -0.6676189771433656] 40%|████ | 40/100 [10:23<14:52, 14.87s/trial, best loss: -0.6676189771433656] 41%|████ | 41/100 [10:32<12:54, 13.12s/trial, best loss: -0.6676189771433656] 42%|████▏ | 42/100 [10:35<09:45, 10.09s/trial, best loss: -0.6676189771433656] 43%|████▎ | 43/100 [11:06<15:33, 16.38s/trial, best loss: -0.6676189771433656] 45%|████▌ | 45/100 [11:25<12:06, 13.22s/trial, best loss: -0.6676189771433656] 46%|████▌ | 46/100 [11:30<10:04, 11.19s/trial, best loss: -0.6676189771433656] 47%|████▋ | 47/100 [11:55<13:05, 14.82s/trial, best loss: -0.6676189771433656] 48%|████▊ | 48/100 [12:03<11:14, 12.97s/trial, best loss: -0.6676189771433656] 49%|████▉ | 49/100 [12:15<10:48, 12.71s/trial, best loss: -0.6676189771433656] 50%|█████ | 50/100 [12:17<08:02, 9.66s/trial, best loss: -0.6676189771433656] 51%|█████ | 51/100 [12:30<08:41, 10.64s/trial, best loss: -0.6676189771433656] 52%|█████▏ | 52/100 [12:34<06:57, 8.70s/trial, best loss: -0.6676189771433656] 53%|█████▎ | 53/100 [12:38<05:44, 7.32s/trial, best loss: -0.6676189771433656] 5

In [0]:
modelo_lgbm_continuo = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto_continuo, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_lgbm_continuo.predict_proba(X_train[aberto_continuo])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_lgbm_continuo.predict_proba(temp_test[aberto_continuo])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (48.41792563169603, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (22.731943746911483, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = aberto_continuo
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_lgbm_continuo, X_train, X_test, y_train, y_test, target, nscores=cols, continuo=aberto_continuo, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.602261,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.397801,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.839268,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.182059,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.792703,1.615136,NaN,2.530711,2.487262,2.958390
5,2.472163,NaN,NaN,2.321931,2.219712,2.280344
6,2.330971,NaN,NaN,1.840165,2.005944,2.096871
7,1.805078,NaN,NaN,1.717734,1.600620,1.815956
8,1.493231,NaN,NaN,1.309091,1.480186,1.231609


### Random Forest

#### Restrito

In [0]:
prefixo = "rf_restrito"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="random_forest", n_estimadores = 100, max_avaliacoes=100, random_state=random_state)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_restrito, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 2%|▏ | 2/100 [00:12<10:10, 6.23s/trial, best loss: -0.6683008909651879] 3%|▎ | 3/100 [00:13<06:32, 4.05s/trial, best loss: -0.6683008909651879] 5%|▌ | 5/100 [00:14<03:21, 2.12s/trial, best loss: -0.6683008909651879] 6%|▌ | 6/100 [00:16<03:16, 2.09s/trial, best loss: -0.6683008909651879] 7%|▋ | 7/100 [00:17<02:45, 1.78s/trial, best loss: -0.6683008909651879] 8%|▊ | 8/100 [00:19<02:50, 1.85s/trial, best loss: -0.6683008909651879] 10%|█ | 10/100 [00:20<01:54, 1.27s/trial, best loss: -0.6683008909651879] 11%|█ | 11/100 [00:21<01:47, 1.21s/trial, best loss: -0.6683008909651879] 12%|█▏ | 12/100 [00:22<01:41, 1.15s/trial, best loss: -0.6683008909651879] 13%|█▎ | 13/100 [00:23<01:36, 1.11s/trial, best loss: -0.6689789210243143] 15%|█▌ | 15/100 [00:25<01:30, 1.07s/trial, best loss: -0.6689789210243143] 16%|█▌ | 16/100 [00:26<01:28, 1.05s/trial, best loss: -0.6689789210243143] 19%|█▉ | 19/100 [00:29<01:23, 1.03s/trial, best loss: -0.6689789210243143] 22%|██▏ | 22/100 [00:31<01:08, 1.14trial/s, best loss: -0.6691973151096354] 23%|██▎ | 23/100 [00:34<01:35, 1.24s/trial, best loss: -0.6694080431392885] 25%|██▌ | 25/100 [00:36<01:27, 1.16s/trial, best loss: -0.6694080431392885] 26%|██▌ | 26/100 [00:37<01:23, 1.13s/trial, best loss: -0.6694080431392885] 27%|██▋ | 27/100 [00:39<01:36, 1.32s/trial, best loss: -0.6694080431392885] 28%|██▊ | 28/100 [00:40<01:29, 1.25s/trial, best loss: -0.6694080431392885] 31%|███ | 31/100 [00:41<00:54, 1.26trial/s, best loss: -0.6694080431392885] 32%|███▏ | 32/100 [00:42<00:56, 1.20trial/s, best loss: -0.6694080431392885] 34%|███▍ | 34/100 [00:44<00:59, 1.12trial/s, best loss: -0.6694080431392885] 36%|███▌ | 36/100 [00:47<01:10, 1.10s/trial, best loss: -0.6694080431392885] 39%|███▉ | 39/100 [00:50<01:04, 1.06s/trial, best loss: -0.6694080431392885] 42%|████▏ | 42/100 [00:51<00:45, 1.26trial/s, best loss: -0.6694080431392885] 43%|████▎ | 43/100 [00:54<01:04, 1.13s/trial, best loss: -0.6694080431392885] 44%|████▍ | 44/100 [00:55<01:01, 1.11s/trial, best loss: -0.6694080431392885] 45%|████▌ | 45/100 [00:56<00:59, 1.09s/trial, best loss: -0.6694080431392885] 47%|████▋ | 47/100 [00:57<00:46, 1.15trial/s, best loss: -0.6694080431392885] 52%|█████▏ | 52/100 [01:01<00:40, 1.20trial/s, best loss: -0.6694080431392885] 53%|█████▎ | 53/100 [01:03<00:46, 1.00trial/s, best loss: -0.6694080431392885] 54%|█████▍ | 54/100 [01:04<00:46, 1.00s/trial, best loss: -0.6694080431392885] 55%|█████▌ | 55/100 [01:06<00:53, 1.20s/trial, best loss: -0.6694080431392885] 56%|█████▌ | 56/100 [01:07<00:50, 1.15s/trial, best loss: -0.6694080431392885] 57%|█████▋ | 57/100 [01:08<00:48, 1.12s/trial, best loss: -0.6694080431392885] 58%|█████▊ | 58/100 [01:11<00:56, 1.35s/trial, best loss: -0.6694080431392885] 59%|█████▉ | 59/100 [01:12<00:51, 1.26s/trial, best loss: -0.6694080431392885] 61%|██████ | 61/100 [01:14<00:44, 1.15s/trial, best loss: -0.6694080431392885] 63%|██████▎ | 63/100 [01:17<00:47, 1.29s/trial, best loss: -0.6694080431392885] 64%|██████▍ | 64/100 [01:18<00:44, 1.23s/trial, best loss: -0.6694080431392885] 65%|██████▌ | 65/100 [01:19<00:41, 1.18s/trial, best loss: -0.6694080431392885] 67%|██████▋ | 67/100 [01:22<00:43, 1.31s/trial, best loss: -0.6694080431392885] 69%|██████▉ | 69/100 [01:25<00:42, 1.38s/trial, best loss: -0.6694080431392885] 70%|███████ | 70/100 [01:26<00:39, 1.31s/trial, best loss: -0.6694080431392885] 72%|███████▏ | 72/100 [01:29<00:38, 1.38s/trial, best loss: -0.6694080431392885] 76%|███████▌ | 76/100 [01:31<00:22, 1.07trial/s, best loss: -0.6694229823229803] 77%|███████▋ | 77/100 [01:35<00:32, 1.42s/trial, best loss: -0.6694229823229803] 79%|███████▉ | 79/100 [01:37<00:27, 1.29s/trial, best loss: -0.6694229823229803] 82%|████████▏ | 82/100 [01:42<00:25, 1.44s/trial, best loss: -0.6694229823229803] 86%|████████▌ | 86/100 [01:48<00:20, 1.47s/trial, best loss: -0.6694229823229803] 88%|████████▊ | 88/100 [01:49<00:14, 1.2

In [0]:
modelo_rf = otimizador.modelo
target = 'inad30'

X_train, X_test, y_train, y_test = train_test_split(X_restrito, y, train_size=.7, random_state=random_state)
 
temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_rf.predict_proba(X_train[restrito])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_rf.predict_proba(temp_test[restrito])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (28.83426795977969, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (24.55247519451585, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = restrito
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_rf, X_train, X_test, y_train, y_test, target, nscores=cols, restrito=restrito, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.602261,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.397801,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.839268,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.182059,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.792703,1.615136,NaN,2.530711,2.487262,2.958390
5,2.472163,NaN,NaN,2.321931,2.219712,2.280344
6,2.330971,NaN,NaN,1.840165,2.005944,2.096871
7,1.805078,NaN,NaN,1.717734,1.600620,1.815956
8,1.493231,NaN,NaN,1.309091,1.480186,1.231609


#### Aberto

In [0]:
prefixo = "rf_aberto"
prefixos.append(prefixo)


otimizador = OtimizadorBayesiano(classificador="random_forest", n_estimadores = 100, max_avaliacoes=100, random_state=random_state)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 2%|▏ | 2/100 [00:27<22:11, 13.59s/trial, best loss: -0.6664987442781807] 6%|▌ | 6/100 [00:28<05:48, 3.71s/trial, best loss: -0.6697534954092784] 9%|▉ | 9/100 [00:29<03:21, 2.22s/trial, best loss: -0.6697534954092784] 10%|█ | 10/100 [00:38<05:06, 3.41s/trial, best loss: -0.6697534954092784] 13%|█▎ | 13/100 [00:44<04:05, 2.82s/trial, best loss: -0.6697534954092784] 16%|█▌ | 16/100 [00:45<02:37, 1.88s/trial, best loss: -0.6697534954092784] 18%|█▊ | 18/100 [00:46<02:04, 1.51s/trial, best loss: -0.6697534954092784] 19%|█▉ | 19/100 [00:48<02:09, 1.59s/trial, best loss: -0.6697534954092784] 20%|██ | 20/100 [00:49<01:58, 1.49s/trial, best loss: -0.6697534954092784] 21%|██ | 21/100 [00:54<02:56, 2.23s/trial, best loss: -0.6697534954092784] 22%|██▏ | 22/100 [00:58<03:28, 2.68s/trial, best loss: -0.6697534954092784] 25%|██▌ | 25/100 [00:59<01:53, 1.51s/trial, best loss: -0.6697534954092784] 28%|██▊ | 28/100 [01:00<01:13, 1.03s/trial, best loss: -0.6700176834996111] 29%|██▉ | 29/100 [01:07<02:23, 2.02s/trial, best loss: -0.6700176834996111] 30%|███ | 30/100 [01:10<02:34, 2.21s/trial, best loss: -0.6700176834996111] 31%|███ | 31/100 [01:14<02:59, 2.59s/trial, best loss: -0.6700176834996111] 33%|███▎ | 33/100 [01:15<02:00, 1.80s/trial, best loss: -0.6700176834996111] 37%|███▋ | 37/100 [01:16<01:02, 1.01trial/s, best loss: -0.670539370174975]  38%|███▊ | 38/100 [01:27<02:39, 2.58s/trial, best loss: -0.670539370174975] 41%|████ | 41/100 [01:28<01:38, 1.68s/trial, best loss: -0.670539370174975] 43%|████▎ | 43/100 [01:29<01:17, 1.35s/trial, best loss: -0.670539370174975] 44%|████▍ | 44/100 [01:30<01:12, 1.30s/trial, best loss: -0.670539370174975] 45%|████▌ | 45/100 [01:36<02:01, 2.20s/trial, best loss: -0.670539370174975] 46%|████▌ | 46/100 [01:38<01:56, 2.16s/trial, best loss: -0.670539370174975] 47%|████▋ | 47/100 [01:44<02:42, 3.06s/trial, best loss: -0.670539370174975] 51%|█████ | 51/100 [01:45<01:11, 1.45s/trial, best loss: -0.670539370174975] 53%|█████▎ | 53/100 [01:46<00:55, 1.18s/trial, best loss: -0.670539370174975] 54%|█████▍ | 54/100 [01:53<01:40, 2.19s/trial, best loss: -0.670539370174975] 56%|█████▌ | 56/100 [01:59<01:48, 2.46s/trial, best loss: -0.670539370174975] 58%|█████▊ | 58/100 [02:01<01:23, 1.99s/trial, best loss: -0.670539370174975] 59%|█████▉ | 59/100 [02:02<01:14, 1.81s/trial, best loss: -0.670539370174975] 60%|██████ | 60/100 [02:05<01:14, 1.85s/trial, best loss: -0.670539370174975] 62%|██████▏ | 62/100 [02:08<01:05, 1.72s/trial, best loss: -0.670539370174975] 66%|██████▌ | 66/100 [03:02<04:26, 7.83s/trial, best loss: -0.670539370174975] 67%|██████▋ | 67/100 [03:08<04:09, 7.55s/trial, best loss: -0.670539370174975] 69%|██████▉ | 69/100 [03:10<02:51, 5.53s/trial, best loss: -0.670539370174975] 70%|███████ | 70/100 [03:15<02:43, 5.44s/trial, best loss: -0.670539370174975] 73%|███████▎ | 73/100 [03:16<01:26, 3.21s/trial, best loss: -0.670539370174975] 75%|███████▌ | 75/100 [03:17<01:00, 2.42s/trial, best loss: -0.670539370174975] 76%|███████▌ | 76/100 [03:18<00:52, 2.17s/trial, best loss: -0.670539370174975] 77%|███████▋ | 77/100 [03:19<00:44, 1.94s/trial, best loss: -0.670539370174975] 79%|███████▉ | 79/100 [03:23<00:41, 1.96s/trial, best loss: -0.670539370174975] 81%|████████ | 81/100 [03:24<00:27, 1.46s/trial, best loss: -0.670539370174975] 82%|████████▏ | 82/100 [03:27<00:31, 1.77s/trial, best loss: -0.670539370174975] 83%|████████▎ | 83/100 [03:31<00:38, 2.26s/trial, best loss: -0.670539370174975] 84%|████████▍ | 84/100 [03:32<00:31, 1.96s/trial, best loss: -0.670539370174975] 85%|████████▌ | 85/100 [03:33<00:25, 1.71s/trial, best loss: -0.670539370174975] 86%|████████▌ | 86/100 [03:34<00:21, 1.53s/trial, best loss: -0.670539370174975] 89%|████████▉ | 89/100 [03:40<00:19, 1.78s/trial, best loss: -0.670539370174975] 90%|█████████ | 90/100 [03:43<00:20, 2.03s/trial, best loss: -0.670539370174975] 91%|█████████ | 91/100 [03:47

In [0]:
modelo_rf_aberto = otimizador.modelo
target = 'inad30'

X_train, X_test, y_train, y_test = train_test_split(X_aberto, y, train_size=.7, random_state=random_state)
 
temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_rf_aberto.predict_proba(X_train[aberto])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_rf_aberto.predict_proba(temp_test[aberto])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (30.052361141167495, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (25.158639915224395, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = aberto
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_rf_aberto, X_train, X_test, y_train, y_test, target, nscores=cols, aberto=aberto, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.602261,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.397801,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.839268,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.182059,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.792703,1.615136,NaN,2.530711,2.487262,2.958390
5,2.472163,NaN,NaN,2.321931,2.219712,2.280344
6,2.330971,NaN,NaN,1.840165,2.005944,2.096871
7,1.805078,NaN,NaN,1.717734,1.600620,1.815956
8,1.493231,NaN,NaN,1.309091,1.480186,1.231609


#### Aberto Contínuo

In [0]:
prefixo = "rf_aberto_continuo"
prefixos.append(prefixo)

otimizador = OtimizadorBayesiano(classificador="random_forest", n_estimadores = 100, random_state=random_state, max_avaliacoes=100)
melhores_params, tentativas = otimizador.otimiza_hiperparams(X_aberto_continuo, y, tam_teste=.3)

0%| | 0/100 [00:00<?, ?trial/s, best loss=?] 1%| | 1/100 [00:14<23:12, 14.07s/trial, best loss: -0.6681378628670197] 2%|▏ | 2/100 [00:15<10:25, 6.38s/trial, best loss: -0.6681378628670197] 5%|▌ | 5/100 [00:17<03:37, 2.29s/trial, best loss: -0.6681378628670197] 6%|▌ | 6/100 [00:20<03:54, 2.50s/trial, best loss: -0.6687410222845483] 7%|▋ | 7/100 [00:21<03:15, 2.10s/trial, best loss: -0.6687410222845483] 8%|▊ | 8/100 [00:24<03:36, 2.35s/trial, best loss: -0.6687410222845483] 11%|█ | 11/100 [00:25<01:52, 1.26s/trial, best loss: -0.6687410222845483] 12%|█▏ | 12/100 [00:29<02:40, 1.83s/trial, best loss: -0.6687410222845483] 13%|█▎ | 13/100 [00:31<02:42, 1.87s/trial, best loss: -0.6687410222845483] 14%|█▍ | 14/100 [00:32<02:22, 1.66s/trial, best loss: -0.6687410222845483] 15%|█▌ | 15/100 [00:34<02:28, 1.75s/trial, best loss: -0.6687410222845483] 17%|█▋ | 17/100 [00:35<01:41, 1.22s/trial, best loss: -0.6694824059140941] 18%|█▊ | 18/100 [00:36<01:36, 1.17s/trial, best loss: -0.6694824059140941] 19%|█▉ | 19/100 [00:38<01:51, 1.38s/trial, best loss: -0.6694824059140941] 22%|██▏ | 22/100 [00:39<01:05, 1.19trial/s, best loss: -0.6694824059140941] 23%|██▎ | 23/100 [00:41<01:22, 1.07s/trial, best loss: -0.6694824059140941] 24%|██▍ | 24/100 [00:43<01:37, 1.28s/trial, best loss: -0.6694824059140941] 25%|██▌ | 25/100 [00:46<02:07, 1.70s/trial, best loss: -0.6694824059140941] 27%|██▋ | 27/100 [00:49<01:58, 1.62s/trial, best loss: -0.6694824059140941] 31%|███ | 31/100 [00:53<01:26, 1.25s/trial, best loss: -0.6694824059140941] 33%|███▎ | 33/100 [00:54<01:10, 1.05s/trial, best loss: -0.6694824059140941] 36%|███▌ | 36/100 [00:55<00:50, 1.27trial/s, best loss: -0.6694824059140941] 38%|███▊ | 38/100 [00:56<00:44, 1.40trial/s, best loss: -0.6694824059140941] 39%|███▉ | 39/100 [00:57<00:46, 1.32trial/s, best loss: -0.6694824059140941] 41%|████ | 41/100 [00:59<00:49, 1.20trial/s, best loss: -0.6694824059140941] 42%|████▏ | 42/100 [01:03<01:22, 1.42s/trial, best loss: -0.6694824059140941] 43%|████▎ | 43/100 [01:06<01:39, 1.75s/trial, best loss: -0.6694824059140941] 44%|████▍ | 44/100 [01:07<01:28, 1.58s/trial, best loss: -0.6694824059140941] 47%|████▋ | 47/100 [01:08<00:51, 1.04trial/s, best loss: -0.6694824059140941] 48%|████▊ | 48/100 [01:09<00:50, 1.03trial/s, best loss: -0.6694824059140941] 50%|█████ | 50/100 [01:10<00:40, 1.24trial/s, best loss: -0.6694824059140941] 51%|█████ | 51/100 [01:11<00:41, 1.18trial/s, best loss: -0.6694824059140941] 52%|█████▏ | 52/100 [01:13<00:53, 1.11s/trial, best loss: -0.6694824059140941] 53%|█████▎ | 53/100 [01:14<00:50, 1.08s/trial, best loss: -0.6694824059140941] 54%|█████▍ | 54/100 [01:15<00:48, 1.06s/trial, best loss: -0.6694824059140941] 55%|█████▌ | 55/100 [01:16<00:47, 1.05s/trial, best loss: -0.6694824059140941] 56%|█████▌ | 56/100 [01:17<00:45, 1.04s/trial, best loss: -0.6694824059140941] 57%|█████▋ | 57/100 [01:18<00:44, 1.03s/trial, best loss: -0.6694824059140941] 58%|█████▊ | 58/100 [01:19<00:42, 1.02s/trial, best loss: -0.6694824059140941] 60%|██████ | 60/100 [01:21<00:40, 1.02s/trial, best loss: -0.6694824059140941] 65%|██████▌ | 65/100 [01:22<00:17, 1.95trial/s, best loss: -0.6694824059140941] 66%|██████▌ | 66/100 [01:25<00:30, 1.12trial/s, best loss: -0.6694824059140941] 67%|██████▋ | 67/100 [01:27<00:36, 1.12s/trial, best loss: -0.6694824059140941] 68%|██████▊ | 68/100 [01:30<00:48, 1.50s/trial, best loss: -0.6694824059140941] 71%|███████ | 71/100 [01:32<00:32, 1.12s/trial, best loss: -0.6694824059140941] 75%|███████▌ | 75/100 [01:33<00:17, 1.40trial/s, best loss: -0.6694824059140941] 76%|███████▌ | 76/100 [01:34<00:18, 1.33trial/s, best loss: -0.6694824059140941] 78%|███████▊ | 78/100 [01:35<00:14, 1.47trial/s, best loss: -0.6694824059140941] 79%|███████▉ | 79/100 [01:37<00:19, 1.10trial/s, best loss: -0.6694824059140941] 80%|████████ | 80/100 [01:38<00:18, 1.08trial/s, best loss: -0.6694824059140941] 81%|████████ | 81/100 [01:42<00:30, 1.61s/trial, 

In [0]:
modelo_rf_continuo = otimizador.modelo
target = 'inad30'
X_train, X_test, y_train, y_test = train_test_split(X_aberto_continuo, y, train_size=.7, random_state=random_state)

temp_train=X_train.join(y_train,rsuffix='r')
temp_train[f'yhat_{prefixo}']=modelo_rf_continuo.predict_proba(X_train[aberto_continuo])[:,1]
print("KS treino: " , calc_perf(temp_train,f'yhat_{prefixo}', target))
  
temp_test=X_test.join(y_test,rsuffix='r')
temp_test[f'yhat_{prefixo}']=modelo_rf_continuo.predict_proba(temp_test[aberto_continuo])[:,1]
print("KS teste: " , calc_perf(temp_test,f'yhat_{prefixo}', target))

KS treino: (31.921044939409366, 64003, 3.2514100901520244, 61922, 2081)
KS teste: (25.114809799246768, 27430, 3.131607728764127, 26571, 859)

In [0]:
cols = aberto_continuo
scores = [prefixo, *cols]

tab_KS = calcula_KS(modelo_rf_continuo, X_train, X_test, y_train, y_test, target, nscores=cols, continuo=aberto_continuo, prefixo=prefixo, safras=safras_dev, base_pd=base_ncli_pd, base_cat=base_ncli_cat)
tab_KS

vis.tabela_qs(modelos, "Safra_main", safras_dev)
vis.tabela_qs(modelos, "Safra_main", safras_nov)

col_0,ab_restrito,WoE_score_ajust,WoE_PF_CP_VOL04_PF,P4_BVS,QuodScore_v2,SCORE_HSPN
qs_ab_restrito,,,,,,
0,4.602261,3.476888,3.841820,5.948860,5.521392,2.220533
1,3.397801,2.931753,2.194145,3.995300,3.875293,4.681586
2,3.839268,2.465887,1.972633,3.015584,3.337041,3.864640
3,3.182059,1.984585,1.787223,3.226229,3.026580,3.404341
4,2.792703,1.615136,NaN,2.530711,2.487262,2.958390
5,2.472163,NaN,NaN,2.321931,2.219712,2.280344
6,2.330971,NaN,NaN,1.840165,2.005944,2.096871
7,1.805078,NaN,NaN,1.717734,1.600620,1.815956
8,1.493231,NaN,NaN,1.309091,1.480186,1.231609


### Validação

In [0]:
target = 'inad30'
modelo_logistica = "yhat_blend"

cols0 = [prefixo for prefixo in prefixos]
cols = [*cols0, modelo_logistica, 'P4_BVS','SCORE_HSPN','QuodScore_v2']
safras_ks = [safra for safra in pd.DatetimeIndex(base_ncli_pd["Safra_main"].unique()) if safra.year < 2022 and safra.month <= 12]

vis.tabela_ks_por_safra(target, modelos=cols, safras=safras_ks)

modelo,P4_BVS,QuodScore_v2,SCORE_HSPN,ab_aberto,ab_aberto_continuo,ab_restrito,lgbm_aberto,lgbm_aberto_continuo,lgbm_restrito,rf_aberto,rf_aberto_continuo,rf_restrito,xg_aberto,xg_aberto_continuo,xg_restrito,yhat_blend
Safra,,,,,,,,,,,,,,,,
2021-08-01,20.2280,18.2180,20.0644,25.3759,26.2707,25.4723,34.2845,41.4749,29.3013,27.9166,28.4441,27.0638,33.7887,33.2590,37.0139,24.0511
2021-09-01,21.7427,20.0108,21.5356,26.2524,27.2020,26.4372,34.8029,40.3720,28.5018,29.6200,30.8197,27.9124,34.8216,34.3296,35.7794,25.1830
2021-10-01,21.1807,20.3182,19.1208,27.5967,27.6130,27.3248,36.4912,41.6276,28.6765,28.8542,30.0575,28.0000,35.0931,35.3719,37.1319,25.7289
2021-11-01,21.2514,17.1391,21.2569,24.3540,23.8295,24.1957,22.7149,21.6950,23.0374,24.1660,24.7292,25.2314,19.7537,23.7916,19.5005,25.7772
2021-12-01,14.6073,15.1148,17.1131,19.4926,21.5674,18.8519,17.1006,16.7575,18.7230,19.7972,21.0817,20.8713,14.6590,21.3726,15.4681,19.4497


### Gráficos

#### Ks por safra

In [0]:
graficos = vis.ks_por_safra(cols, safras_dev, agrupar=True)

--------------------------------------------------------------------------- 
 KeyError Traceback (most recent call last)
 <command-3592731989734048> in <module> 
 ----> 1 graficos = vis . ks_por_safra ( cols , safras_dev ) 

 <command-1731702899285260> in ks_por_safra (self, modelos, safras, col_target, agrupar, kind, title, ylabel, figsize, grid, tab_KS_pivot) 
 101 if not agrupar : 
 102 modelos = [ * modelos , col_target ] 
 --> 103 grafico = self . tab_KS_pivot [ modelos ] . plot ( kind = kind , title = f"{title}" , figsize = figsize , marker = '.' , markersize = 16 ) 
 104 
 105 plt . ylabel ( "KS" ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/frame.py in __getitem__ (self, key) 
 2804 if is_iterator ( key ) : 
 2805 key = list ( key ) 
 -> 2806 indexer = self . loc . _get_listlike_indexer ( key , axis = 1 , raise_missing = True ) [ 1 ] 
 2807 
 2808 # take() does not accept boolean indexers 

 /databricks/python/lib/python3.7/site-packages/pandas/core/indexing.py in _get_listlike_indexer (self, key, axis, raise_missing) 
 1550 
 1551 self._validate_read_indexer(
 -> 1552 keyarr , indexer , o . _get_axis_number ( axis ) , raise_missing = raise_missing
 1553 )
 1554 return keyarr , indexer

 /databricks/python/lib/python3.7/site-packages/pandas/core/indexing.py in _validate_read_indexer (self, key, indexer, axis, raise_missing) 
 1643 if not ( self . name == "loc" and not raise_missing ) : 
 1644 not_found = list ( set ( key ) - set ( ax ) ) 
 -> 1645 raise KeyError ( f"{not_found} not in index" ) 
 1646 
 1647 # we skip the warning on Categorical/Interval 

 KeyError : '[None] not in index'

In [0]:
restritos = [modelo for modelo in cols if "restrito" in modelo]
abertos = [modelo for modelo in cols if "aberto" in modelo and "continuo" not in modelo]
continuos = [modelo for modelo in cols if "continuo" in modelo]

restritos_blend = [modelo_logistica, *restritos]
abertos_blend = [modelo_logistica, *abertos]
continuos_blend = [modelo_logistica, *continuos]

In [0]:
vis.tab_KS_pivot[restritos_blend].plot(kind="line", title="KS do modelo por safra (Restrito)", figsize=(8,6), marker='.', markersize=14)
plt.ylabel("KS")
plt.xlabel("Safra - 2021")

In [0]:
vis.tab_KS_pivot[abertos_blend].plot.line(title="KS do modelo por safra (Aberto)", figsize=(8,6), marker='.', markersize=14)
plt.ylabel("KS")
plt.xlabel("Safra - 2021")

In [0]:
vis.tab_KS_pivot[continuos_blend].plot.line(title="KS do modelo  por safra (Contínuo)", figsize=(8,6), marker='.', markersize=14)
plt.ylabel("KS")
plt.xlabel("Safra - 2021")

In [0]:
atualizados = [modelo_logistica, "xg_restrito"]
ax_res = vis.tab_KS_pivot[atualizados].plot.line( title="KS de cada modelo por safra", figsize=(10,8), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[40,])

In [0]:
ax_res = vis.tab_KS_pivot[restritos_blend].plot.line( title="KS de cada modelo por safra", figsize=(10,8), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[40,])

In [0]:
ax_res = vis.tab_KS_pivot[abertos_blend].plot.line( title="KS de cada modelo por safra", figsize=(10,8), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[40,])

In [0]:
ax_res = vis.tab_KS_pivot[continuos_blend].plot.line( title="KS de cada modelo por safra", figsize=(10,8), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[40,])

In [0]:
atualizados = [modelo_logistica, "lgbm_aberto"]
vis.tab_KS_pivot[atualizados].plot.line(title="KS de cada modelo por safra", figsize=(10,10), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\

In [0]:
atualizados = [modelo_logistica, "lgbm_aberto_continuo"]
vis.tab_KS_pivot[atualizados].plot.line(title="KS de cada modelo por safra", figsize=(10,10), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\

In [0]:
atualizados_final = [modelo_logistica, "lgbm_aberto", "lgbm_aberto_continuo", "xg_restrito"]
vis.tab_KS_pivot[atualizados_final].plot.line(title="KS de cada modelo por safra", figsize=(10,6), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\

####Decis Inad

In [0]:
prefixos = list(dict.fromkeys(prefixos))
 
modelos = [*prefixos, modelo_logistica, 'P4_BVS', 'SCORE_HSPN', 'QuodScore_v2']

tab_decis = vis.tabela_qs(modelos)
tab_decis

In [0]:
tab_decis[restritos_blend].plot(kind="line", title="Ordenação inad por decil (Restrito)", figsize=(8,6), marker='.', markersize=14 )
plt.xlabel("Decil")
plt.ylabel("%inad30")
plt.legend(title="modelo")

In [0]:
tab_decis[abertos_blend].plot(kind="line", title="Ordenação inad por decil (Aberto)", figsize=(8,6), marker='.', markersize=14 )
plt.xlabel("Decil")
plt.ylabel("%inad30")
plt.legend(title="modelo")

In [0]:
tab_decis[continuos_blend].plot(kind="line", title="Ordenação inad por decil (Contínuo)", figsize=(8,6), marker='.', markersize=14 )
plt.xlabel("Decil")
plt.ylabel("%inad30")
plt.legend(title="modelo")

In [0]:
tab_decis[restritos_blend].plot.line(title="KS de cada modelo por safra", figsize=(10,10), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\

In [0]:
tab_decis[abertos_blend].plot.line(title="KS de cada modelo por safra", figsize=(10,10), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\

In [0]:
tab_decis[continuos_blend].plot.line(title="KS de cada modelo por safra", figsize=(10,10), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\

In [0]:
atualizados = [modelo_logistica, "ab_aberto", "ab_restrito", "xg_aberto", "xg_aberto_continuo"]
tab_decis[atualizados].plot.line(title="Ordenação inad por decil", figsize=(10,6), marker='.', markersize=14, xlim=[safras_dev[0], safras_dev[-1]], ylim=[20,])\
.legend(loc='upper left',title='modelo')